In [ ]:
import sys
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/poc")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/plugins")
sys.path.append("/home/tsevero/notebooks/SAT_BIG_DATA/data-pipeline/batch/dags")

#Import libs python
from pyspark.sql.types import *
from pyspark.sql.functions import *
from datetime import date

#Import libs internas
from utils import spark_utils_session as utils

from hooks.hdfs.hdfs_helper import HdfsHelper
from jobs.job_base_config import BaseETLJobClass

import poc_helper
poc_helper.load_env("PROD")

In [ ]:
def get_session(profile: str, dynamic_allocation_enabled: bool = True) -> utils.DBASparkAppSession:
    """Generates DBASparkAppSession."""
    
    app_name = "tsevero_fisca"
    
    spark_builder = (utils.DBASparkAppSession
                     .builder
                     .setAppName(app_name)
                     .usingProcessProfile(profile)
                    )
    
    if dynamic_allocation_enabled:
        spark_builder.autoResourceManagement()

    return spark_builder.build()

session = get_session(profile='efd_t2')

In [ ]:
session.sparkSession.sql("SHOW DATABASES").show(truncate=False)

In [ ]:
# ============================================================================
# CÉLULA 1: SETUP E IMPORTS
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    roc_curve, auc, silhouette_score,
    mean_squared_error, r2_score
)
import xgboost as xgb

# PySpark imports com aliases
from pyspark.sql.functions import (
    col as spark_col,
    sum as spark_sum,
    avg as spark_avg,
    count as spark_count,
    max as spark_max,
    min as spark_min,
    when as spark_when,
    desc as spark_desc,
    asc as spark_asc,
    round as spark_round,
    coalesce as spark_coalesce
)

# Acesso ao SparkSession
spark = session.sparkSession

# Configurações visuais
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Imports realizados com sucesso!")
print(f"📦 Pandas: {pd.__version__}")
print(f"📦 NumPy: {np.__version__}")

In [ ]:
# ============================================================================
# CÉLULA 2: FUNÇÕES AUXILIARES
# ============================================================================

def load_spark_to_pandas_safe(query, limit=100000, view_name="temp_view"):
    """
    Carrega dados do Spark para Pandas de forma segura
    """
    try:
        # Criar view temporária
        spark.sql(f"CREATE OR REPLACE TEMPORARY VIEW {view_name} AS {query}")
        
        # Verificar tamanho
        total = spark.sql(f"SELECT COUNT(*) as total FROM {view_name}").collect()[0]['total']
        print(f"📊 Total de registros encontrados: {total:,}")
        
        if total == 0:
            print("⚠️ Nenhum registro encontrado!")
            return pd.DataFrame()
        
        elif total > limit:
            print(f"⚠️ Muitos registros ({total:,}), limitando a {limit:,}...")
            df_pandas = spark.sql(f"SELECT * FROM {view_name} LIMIT {limit}").toPandas()
        
        else:
            df_spark = spark.sql(f"SELECT * FROM {view_name}")
            df_spark.cache()
            df_pandas = df_spark.toPandas()
            df_spark.unpersist()
        
        print(f"✅ DataFrame carregado: {df_pandas.shape[0]:,} linhas x {df_pandas.shape[1]} colunas")
        return df_pandas
    
    except Exception as e:
        print(f"❌ Erro ao carregar dados: {str(e)}")
        return pd.DataFrame()


def format_currency(value):
    """Formata valor em moeda brasileira"""
    return f"R$ {value:,.2f}".replace(',', 'X').replace('.', ',').replace('X', '.')


def format_percentage(value):
    """Formata percentual"""
    return f"{value:.2f}%"


def create_metric_card(value, title, suffix="", prefix=""):
    """Cria card de métrica para visualização"""
    return f"""
    <div style='text-align: center; padding: 20px; background-color: #f0f2f6; border-radius: 10px;'>
        <h3 style='color: #0e1117; margin: 0;'>{title}</h3>
        <h1 style='color: #ff4b4b; margin: 10px 0;'>{prefix}{value:,.2f}{suffix}</h1>
    </div>
    """


print("✅ Funções auxiliares carregadas!")

In [ ]:
# ============================================================================
# CÉLULA 3: CARREGAMENTO DASHBOARD EXECUTIVO
# ============================================================================

query_dashboard = """
SELECT 
    ano,
    CAST(COALESCE(qtd_acompanhamentos, 0) AS DOUBLE) AS qtd_acompanhamentos,
    CAST(COALESCE(empresas_acompanhadas, 0) AS DOUBLE) AS empresas_acompanhadas,
    CAST(COALESCE(qtd_infracoes_lavradas, 0) AS DOUBLE) AS qtd_infracoes_lavradas,
    CAST(COALESCE(empresas_fiscalizadas, 0) AS DOUBLE) AS empresas_fiscalizadas,
    CAST(COALESCE(infracoes_com_ciencia, 0) AS DOUBLE) AS infracoes_com_ciencia,
    CAST(COALESCE(valor_total_infracoes, 0) AS DOUBLE) AS valor_total_infracoes,
    CAST(COALESCE(valor_imposto_infracoes, 0) AS DOUBLE) AS valor_imposto_infracoes,
    CAST(COALESCE(valor_multa_infracoes, 0) AS DOUBLE) AS valor_multa_infracoes,
    CAST(COALESCE(valor_juros_infracoes, 0) AS DOUBLE) AS valor_juros_infracoes,
    CAST(COALESCE(qtd_nfs_emitidas, 0) AS DOUBLE) AS qtd_nfs_emitidas,
    CAST(COALESCE(valor_total_nfs, 0) AS DOUBLE) AS valor_total_nfs,
    CAST(COALESCE(valor_imposto_nfs, 0) AS DOUBLE) AS valor_imposto_nfs,
    CAST(COALESCE(valor_multa_nfs, 0) AS DOUBLE) AS valor_multa_nfs,
    CAST(COALESCE(valor_juros_nfs, 0) AS DOUBLE) AS valor_juros_nfs,
    CAST(COALESCE(qtd_encerramentos, 0) AS DOUBLE) AS qtd_encerramentos,
    CAST(COALESCE(qtd_encerramentos_com_resultado, 0) AS DOUBLE) AS qtd_encerramentos_com_resultado,
    CAST(COALESCE(qtd_ciclos_completos, 0) AS DOUBLE) AS qtd_ciclos_completos,
    CAST(COALESCE(media_dias_infracao_nf, 0) AS DOUBLE) AS media_dias_infracao_nf,
    CAST(COALESCE(media_dias_infracao_encerramento, 0) AS DOUBLE) AS media_dias_infracao_encerramento,
    CAST(COALESCE(taxa_conversao_infracao_nf, 0) AS DOUBLE) AS taxa_conversao_infracao_nf,
    CAST(COALESCE(valor_medio_infracao, 0) AS DOUBLE) AS valor_medio_infracao,
    CAST(COALESCE(valor_medio_nf, 0) AS DOUBLE) AS valor_medio_nf,
    CAST(COALESCE(qtd_afres_ativos, 0) AS DOUBLE) AS qtd_afres_ativos,
    CAST(COALESCE(media_infracoes_por_afre, 0) AS DOUBLE) AS media_infracoes_por_afre,
    CAST(COALESCE(valor_medio_por_afre, 0) AS DOUBLE) AS valor_medio_por_afre
FROM teste.fisca_dashboard_executivo
ORDER BY ano DESC
"""

df_dashboard = load_spark_to_pandas_safe(query_dashboard, view_name="vw_dashboard")

# Exibir resumo
if not df_dashboard.empty:
    print("\n📊 RESUMO DO DASHBOARD EXECUTIVO")
    print("=" * 80)
    print(f"Período: {df_dashboard['ano'].min()} a {df_dashboard['ano'].max()}")
    print(f"Total de Infrações: {df_dashboard['qtd_infracoes_lavradas'].sum():,.0f}")
    print(f"Total de NFs: {df_dashboard['qtd_nfs_emitidas'].sum():,.0f}")
    print(f"Valor Total Infrações: {format_currency(df_dashboard['valor_total_infracoes'].sum())}")
    print(f"Valor Total NFs: {format_currency(df_dashboard['valor_total_nfs'].sum())}")
    print(f"Taxa Conversão Média: {df_dashboard['taxa_conversao_infracao_nf'].mean():.2f}%")

In [ ]:
# ============================================================================
# CÉLULA 4: EDA - ANÁLISE EXPLORATÓRIA DASHBOARD EXECUTIVO
# ============================================================================

if not df_dashboard.empty:
    
    print("📊 ESTATÍSTICAS DESCRITIVAS")
    print("=" * 80)
    
    # Estatísticas principais
    stats_cols = [
        'qtd_infracoes_lavradas', 'qtd_nfs_emitidas', 'valor_total_infracoes',
        'valor_total_nfs', 'taxa_conversao_infracao_nf', 'media_dias_infracao_nf'
    ]
    
    for col in stats_cols:
        if col in df_dashboard.columns:
            print(f"\n{col}:")
            print(f"  Média: {df_dashboard[col].mean():,.2f}")
            print(f"  Mediana: {df_dashboard[col].median():,.2f}")
            print(f"  Desvio Padrão: {df_dashboard[col].std():,.2f}")
            print(f"  Min: {df_dashboard[col].min():,.2f}")
            print(f"  Max: {df_dashboard[col].max():,.2f}")
    
    # Verificar valores nulos
    print("\n\n📋 VALORES NULOS POR COLUNA")
    print("=" * 80)
    null_counts = df_dashboard.isnull().sum()
    if null_counts.sum() > 0:
        print(null_counts[null_counts > 0])
    else:
        print("✅ Nenhum valor nulo encontrado!")
    
    # Correlações
    print("\n\n🔗 CORRELAÇÕES PRINCIPAIS")
    print("=" * 80)
    
    numeric_cols = df_dashboard.select_dtypes(include=[np.number]).columns
    corr_matrix = df_dashboard[numeric_cols].corr()
    
    # Top 10 correlações
    corr_pairs = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            corr_pairs.append({
                'var1': corr_matrix.columns[i],
                'var2': corr_matrix.columns[j],
                'correlation': corr_matrix.iloc[i, j]
            })
    
    corr_df = pd.DataFrame(corr_pairs).sort_values('correlation', ascending=False)
    print("\nTop 10 Correlações Positivas:")
    print(corr_df.head(10))
    
    print("\nTop 10 Correlações Negativas:")
    print(corr_df.tail(10))

else:
    print("⚠️ DataFrame vazio - pulando análise exploratória")

In [ ]:
# ============================================================================
# CÉLULA 5: VISUALIZAÇÃO - EVOLUÇÃO TEMPORAL
# ============================================================================

if not df_dashboard.empty:
    
    # Criar figura com subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Evolução de Infrações e NFs',
            'Evolução dos Valores (R$ Milhões)',
            'Taxa de Conversão ao Longo do Tempo',
            'Produtividade por AFRE'
        ),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )
    
    # 1. Infrações e NFs
    fig.add_trace(
        go.Bar(
            x=df_dashboard['ano'],
            y=df_dashboard['qtd_infracoes_lavradas'],
            name='Infrações Lavradas',
            marker_color='indianred'
        ),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(
            x=df_dashboard['ano'],
            y=df_dashboard['qtd_nfs_emitidas'],
            name='NFs Emitidas',
            marker_color='lightsalmon'
        ),
        row=1, col=1
    )
    
    # 2. Valores
    fig.add_trace(
        go.Scatter(
            x=df_dashboard['ano'],
            y=df_dashboard['valor_total_infracoes'] / 1_000_000,
            name='Valor Infrações',
            mode='lines+markers',
            line=dict(color='blue', width=3)
        ),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=df_dashboard['ano'],
            y=df_dashboard['valor_total_nfs'] / 1_000_000,
            name='Valor NFs',
            mode='lines+markers',
            line=dict(color='green', width=3)
        ),
        row=1, col=2
    )
    
    # 3. Taxa de Conversão
    fig.add_trace(
        go.Scatter(
            x=df_dashboard['ano'],
            y=df_dashboard['taxa_conversao_infracao_nf'],
            name='Taxa Conversão',
            mode='lines+markers',
            line=dict(color='purple', width=3),
            fill='tozeroy'
        ),
        row=2, col=1
    )
    
    # 4. Produtividade
    fig.add_trace(
        go.Bar(
            x=df_dashboard['ano'],
            y=df_dashboard['media_infracoes_por_afre'],
            name='Infrações/AFRE',
            marker_color='teal'
        ),
        row=2, col=2
    )
    
    # Layout
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="📊 Dashboard Executivo - Evolução Temporal FISCA",
        title_font_size=20
    )
    
    fig.update_xaxes(title_text="Ano", row=2, col=1)
    fig.update_xaxes(title_text="Ano", row=2, col=2)
    fig.update_yaxes(title_text="Quantidade", row=1, col=1)
    fig.update_yaxes(title_text="R$ Milhões", row=1, col=2)
    fig.update_yaxes(title_text="%", row=2, col=1)
    fig.update_yaxes(title_text="Infrações/AFRE", row=2, col=2)
    
    fig.show()
    
    print("✅ Gráfico de evolução temporal criado!")

else:
    print("⚠️ DataFrame vazio - pulando visualização")

In [ ]:
# ============================================================================
# CÉLULA 6: CARREGAMENTO FISCALIZAÇÕES CONSOLIDADAS
# ============================================================================

query_fiscalizacoes = """
SELECT 
    cnpj,
    nm_razao_social,
    regime_tributario,
    municipio,
    uf,
    cnae_secao,
    cnae_secao_descricao,
    ano_infracao,
    CAST(COALESCE(valor_total_infracao, 0) AS DOUBLE) AS valor_total_infracao,
    CAST(COALESCE(valor_imposto_infracao, 0) AS DOUBLE) AS valor_imposto_infracao,
    CAST(COALESCE(valor_multa_infracao, 0) AS DOUBLE) AS valor_multa_infracao,
    CAST(COALESCE(valor_total_nf, 0) AS DOUBLE) AS valor_total_nf,
    CAST(COALESCE(gerou_notificacao, 0) AS INT) AS gerou_notificacao,
    CAST(COALESCE(teve_encerramento, 0) AS INT) AS teve_encerramento,
    CAST(COALESCE(ciclo_completo, 0) AS INT) AS ciclo_completo,
    CAST(COALESCE(dias_infracao_ate_nf, 0) AS DOUBLE) AS dias_infracao_ate_nf,
    CAST(COALESCE(pagou_dp_infracao, 0) AS INT) AS pagou_dp_infracao,
    situacao_final
FROM teste.fisca_fiscalizacoes_consolidadas
WHERE ano_infracao >= 2020
"""

df_fisc = load_spark_to_pandas_safe(
    query_fiscalizacoes, 
    limit=150000,
    view_name="vw_fiscalizacoes"
)

if not df_fisc.empty:
    print("\n📊 RESUMO FISCALIZAÇÕES CONSOLIDADAS")
    print("=" * 80)
    print(f"Total de registros: {len(df_fisc):,}")
    print(f"Empresas únicas: {df_fisc['cnpj'].nunique():,}")
    print(f"Valor total infrações: {format_currency(df_fisc['valor_total_infracao'].sum())}")
    print(f"Valor total NFs: {format_currency(df_fisc['valor_total_nf'].sum())}")
    print(f"\nDistribuição por Situação:")
    print(df_fisc['situacao_final'].value_counts())

In [ ]:
# ============================================================================
# CÉLULA 7: EDA - ANÁLISE POR SETOR (CNAE)
# ============================================================================

if not df_fisc.empty:
    
    print("📊 ANÁLISE POR SETOR ECONÔMICO (CNAE)")
    print("=" * 80)
    
    # Agrupar por seção CNAE
    setor_analysis = df_fisc.groupby('cnae_secao_descricao').agg({
        'cnpj': 'count',
        'valor_total_infracao': 'sum',
        'valor_total_nf': 'sum',
        'gerou_notificacao': 'sum',
        'ciclo_completo': 'sum',
        'dias_infracao_ate_nf': 'mean'
    }).reset_index()
    
    setor_analysis.columns = [
        'setor', 'qtd_fiscalizacoes', 'valor_infracoes', 
        'valor_nfs', 'qtd_nfs', 'ciclos_completos', 'media_dias'
    ]
    
    # Calcular taxa de conversão
    setor_analysis['taxa_conversao'] = (
        setor_analysis['qtd_nfs'] / setor_analysis['qtd_fiscalizacoes'] * 100
    )
    
    # Ordenar por valor
    setor_analysis = setor_analysis.sort_values('valor_nfs', ascending=False)
    
    print("\nTop 10 Setores por Valor de NFs:")
    print(setor_analysis.head(10)[['setor', 'qtd_fiscalizacoes', 'valor_nfs', 'taxa_conversao']])
    
    # Visualização - Top 15 setores
    top_setores = setor_analysis.head(15)
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Valor de NFs por Setor (Top 15)', 'Taxa de Conversão por Setor (Top 15)'),
        specs=[[{"type": "bar"}, {"type": "bar"}]]
    )
    
    # Gráfico 1: Valores
    fig.add_trace(
        go.Bar(
            y=top_setores['setor'],
            x=top_setores['valor_nfs'] / 1_000_000,
            orientation='h',
            marker_color='steelblue',
            name='Valor NFs'
        ),
        row=1, col=1
    )
    
    # Gráfico 2: Taxa de conversão
    fig.add_trace(
        go.Bar(
            y=top_setores['setor'],
            x=top_setores['taxa_conversao'],
            orientation='h',
            marker_color='coral',
            name='Taxa Conversão'
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        height=600,
        title_text="📊 Análise por Setor Econômico",
        showlegend=False
    )
    
    fig.update_xaxes(title_text="R$ Milhões", row=1, col=1)
    fig.update_xaxes(title_text="% Conversão", row=1, col=2)
    
    fig.show()
    
    print("\n✅ Análise por setor concluída!")

else:
    print("⚠️ DataFrame vazio - pulando análise por setor")

In [ ]:
# ============================================================================
# CÉLULA 8: ANÁLISE GEOGRÁFICA (CORRIGIDA)
# ============================================================================

if not df_fisc.empty:
    
    print("🗺️ ANÁLISE GEOGRÁFICA")
    print("=" * 80)
    
    # Agrupar por município (SEM taxa_conversao no agg)
    geo_analysis = df_fisc.groupby(['municipio', 'uf']).agg({
        'cnpj': 'count',
        'valor_total_infracao': 'sum',
        'valor_total_nf': 'sum',
        'gerou_notificacao': 'sum'
    }).reset_index()
    
    geo_analysis.columns = [
        'municipio', 'uf', 'qtd_fiscalizacoes', 
        'valor_infracoes', 'valor_nfs', 'qtd_nfs'
    ]
    
    # Calcular taxa de conversão DEPOIS do reset_index
    geo_analysis['taxa_conversao'] = (
        geo_analysis['qtd_nfs'] / geo_analysis['qtd_fiscalizacoes'] * 100
    )
    
    # Top 20 municípios
    top_municipios = geo_analysis.nlargest(20, 'valor_nfs')
    
    print("\nTop 20 Municípios por Valor de NFs:")
    print(top_municipios[['municipio', 'uf', 'qtd_fiscalizacoes', 'valor_nfs']])
    
    # Visualização
    fig = px.bar(
        top_municipios,
        x='valor_nfs',
        y='municipio',
        color='uf',
        orientation='h',
        title='Top 20 Municípios por Valor de NFs Emitidas',
        labels={'valor_nfs': 'Valor Total NFs (R$)', 'municipio': 'Município'},
        text='qtd_fiscalizacoes'
    )
    
    fig.update_traces(texttemplate='%{text} fisc.', textposition='outside')
    fig.update_layout(height=700, showlegend=True)
    
    fig.show()
    
    # Mapa de calor - Distribuição por UF
    uf_analysis = df_fisc.groupby('uf').agg({
        'cnpj': 'count',
        'valor_total_nf': 'sum'
    }).reset_index()
    
    fig2 = go.Figure(data=go.Choropleth(
        locations=uf_analysis['uf'],
        z=uf_analysis['valor_total_nf'],
        locationmode='USA-states',
        colorscale='Reds',
        text=uf_analysis['uf'],
        colorbar_title="Valor NFs"
    ))
    
    fig2.update_layout(
        title_text='Distribuição Geográfica - Valor de NFs por UF',
        geo_scope='south america',
        height=500
    )
    
    fig2.show()
    
    print("\n✅ Análise geográfica concluída!")

else:
    print("⚠️ DataFrame vazio - pulando análise geográfica")

In [ ]:
# ============================================================================
# CÉLULA 9: PREPARAÇÃO PARA MACHINE LEARNING
# ============================================================================

if not df_fisc.empty and len(df_fisc) >= 100:
    
    print("🤖 PREPARAÇÃO DOS DADOS PARA MACHINE LEARNING")
    print("=" * 80)
    
    # Criar dataset para ML
    df_ml = df_fisc.copy()
    
    # Remover valores nulos e criar features
    df_ml = df_ml.dropna(subset=[
        'valor_total_infracao', 'regime_tributario', 
        'cnae_secao', 'municipio'
    ])
    
    # Feature Engineering
    df_ml['valor_log'] = np.log1p(df_ml['valor_total_infracao'])
    df_ml['tem_multa'] = (df_ml['valor_multa_infracao'] > 0).astype(int)
    df_ml['perc_multa'] = np.where(
        df_ml['valor_total_infracao'] > 0,
        df_ml['valor_multa_infracao'] / df_ml['valor_total_infracao'] * 100,
        0
    )
    
    # Faixa de valor
    df_ml['faixa_valor'] = pd.cut(
        df_ml['valor_total_infracao'],
        bins=[0, 50000, 100000, 500000, 1000000, np.inf],
        labels=['Muito Baixo', 'Baixo', 'Médio', 'Alto', 'Muito Alto']
    )
    
    # Encoding de variáveis categóricas
    le_regime = LabelEncoder()
    le_cnae = LabelEncoder()
    le_municipio = LabelEncoder()
    le_uf = LabelEncoder()
    
    df_ml['regime_encoded'] = le_regime.fit_transform(df_ml['regime_tributario'].fillna('DESCONHECIDO'))
    df_ml['cnae_encoded'] = le_cnae.fit_transform(df_ml['cnae_secao'].fillna('DESCONHECIDO'))
    df_ml['municipio_encoded'] = le_municipio.fit_transform(df_ml['municipio'].fillna('DESCONHECIDO'))
    df_ml['uf_encoded'] = le_uf.fit_transform(df_ml['uf'].fillna('SC'))
    
    # Selecionar features para modelo
    feature_cols = [
        'valor_log', 'valor_imposto_infracao', 'valor_multa_infracao',
        'tem_multa', 'perc_multa', 'regime_encoded', 'cnae_encoded',
        'municipio_encoded', 'uf_encoded', 'ano_infracao',
        'pagou_dp_infracao'
    ]
    
    # Target: gerou_notificacao
    X = df_ml[feature_cols].fillna(0)
    y = df_ml['gerou_notificacao']
    
    print(f"\n📊 Dataset ML preparado:")
    print(f"  - Features: {X.shape[1]}")
    print(f"  - Amostras: {X.shape[0]:,}")
    print(f"  - Target positivo: {y.sum():,} ({y.mean()*100:.2f}%)")
    print(f"  - Target negativo: {(~y.astype(bool)).sum():,} ({(1-y.mean())*100:.2f}%)")
    
    print("\n✅ Dados preparados para Machine Learning!")
    
    # Salvar para próximas células
    ml_data = {
        'X': X,
        'y': y,
        'df_ml': df_ml,
        'feature_cols': feature_cols,
        'encoders': {
            'regime': le_regime,
            'cnae': le_cnae,
            'municipio': le_municipio,
            'uf': le_uf
        }
    }

else:
    print("⚠️ Dados insuficientes para Machine Learning")
    ml_data = None

In [ ]:
# ============================================================================
# CÉLULA 10: RANDOM FOREST - PREVISÃO DE NOTIFICAÇÃO
# ============================================================================

if ml_data is not None:
    
    print("🌲 RANDOM FOREST - PREVISÃO DE NOTIFICAÇÃO FISCAL")
    print("=" * 80)
    
    X = ml_data['X']
    y = ml_data['y']
    
    # Split train/test
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    print(f"\n📊 Distribuição dos dados:")
    print(f"  Treino: {len(X_train):,} amostras")
    print(f"  Teste: {len(X_test):,} amostras")
    
    # Normalização
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Treinar Random Forest
    print("\n🔄 Treinando Random Forest...")
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        min_samples_split=20,
        min_samples_leaf=10,
        random_state=42,
        n_jobs=-1,
        class_weight='balanced'
    )
    
    rf_model.fit(X_train_scaled, y_train)
    
    # Predições
    y_pred = rf_model.predict(X_test_scaled)
    y_pred_proba = rf_model.predict_proba(X_test_scaled)[:, 1]
    
    # Métricas
    print("\n📊 RESULTADOS DO MODELO:")
    print("=" * 80)
    print(classification_report(y_test, y_pred, target_names=['Não Notificou', 'Notificou']))
    
    # Importância das features
    feature_importance = pd.DataFrame({
        'feature': ml_data['feature_cols'],
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n🔝 Top 10 Features Mais Importantes:")
    print(feature_importance.head(10))
    
    # Visualização - Feature Importance
    fig = px.bar(
        feature_importance.head(15),
        x='importance',
        y='feature',
        orientation='h',
        title='Importância das Features - Random Forest',
        labels={'importance': 'Importância', 'feature': 'Feature'}
    )
    fig.update_layout(height=500)
    fig.show()
    
    # Curva ROC
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    fig_roc = go.Figure()
    fig_roc.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f'ROC (AUC = {roc_auc:.3f})',
        line=dict(color='darkorange', width=2)
    ))
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        name='Random',
        line=dict(color='navy', width=2, dash='dash')
    ))
    
    fig_roc.update_layout(
        title='Curva ROC - Random Forest',
        xaxis_title='Taxa de Falsos Positivos',
        yaxis_title='Taxa de Verdadeiros Positivos',
        height=500
    )
    fig_roc.show()
    
    # Matriz de Confusão
    cm = confusion_matrix(y_test, y_pred)
    fig_cm = px.imshow(
        cm,
        text_auto=True,
        labels=dict(x="Predito", y="Real", color="Quantidade"),
        x=['Não Notificou', 'Notificou'],
        y=['Não Notificou', 'Notificou'],
        title='Matriz de Confusão - Random Forest'
    )
    fig_cm.show()
    
    # Salvar modelo
    rf_results = {
        'model': rf_model,
        'scaler': scaler,
        'feature_importance': feature_importance,
        'auc': roc_auc,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    
    print("\n✅ Random Forest treinado com sucesso!")
    
else:
    print("⚠️ Dados ML não disponíveis")
    rf_results = None

In [ ]:
# ============================================================================
# CÉLULA 11: XGBOOST - PREVISÃO DE NOTIFICAÇÃO FISCAL (CORRIGIDA)
# ============================================================================

if ml_data is not None:
    
    print("🚀 XGBOOST - PREVISÃO DE NOTIFICAÇÃO FISCAL")
    print("=" * 80)
    
    X = ml_data['X']
    y = ml_data['y']
    
    # Split train/test (mesmo split do RF para comparação)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )
    
    # Calcular scale_pos_weight para balanceamento
    scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
    
    print(f"\n📊 Scale pos weight: {scale_pos_weight:.2f}")
    
    # Treinar XGBoost
    print("\n🔄 Treinando XGBoost...")
    
    xgb_model = xgb.XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        n_jobs=-1,
        eval_metric='logloss'
    )
    
    xgb_model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=False
    )
    
    # Predições
    y_pred_xgb = xgb_model.predict(X_test)
    y_pred_proba_xgb = xgb_model.predict_proba(X_test)[:, 1]
    
    # Métricas
    print("\n📊 RESULTADOS DO MODELO XGBOOST:")
    print("=" * 80)
    print(classification_report(y_test, y_pred_xgb, target_names=['Não Notificou', 'Notificou']))
    
    # Importância das features
    feature_importance_xgb = pd.DataFrame({
        'feature': ml_data['feature_cols'],
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\n🔝 Top 10 Features Mais Importantes (XGBoost):")
    print(feature_importance_xgb.head(10))
    
    # Visualização - Feature Importance
    fig = px.bar(
        feature_importance_xgb.head(15),
        x='importance',
        y='feature',
        orientation='h',
        title='Importância das Features - XGBoost',
        labels={'importance': 'Importância', 'feature': 'Feature'},
        color='importance',
        color_continuous_scale='Viridis'
    )
    fig.update_layout(height=500)
    fig.show()
    
    # Curva ROC
    fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_pred_proba_xgb)
    roc_auc_xgb = auc(fpr_xgb, tpr_xgb)
    
    # Comparar com Random Forest
    if rf_results is not None:
        fpr_rf, tpr_rf, _ = roc_curve(y_test, rf_results['probabilities'])
        
        fig_comp = go.Figure()
        fig_comp.add_trace(go.Scatter(
            x=fpr_xgb, y=tpr_xgb,
            mode='lines',
            name=f'XGBoost (AUC = {roc_auc_xgb:.3f})',
            line=dict(color='green', width=2)
        ))
        fig_comp.add_trace(go.Scatter(
            x=fpr_rf, y=tpr_rf,
            mode='lines',
            name=f'Random Forest (AUC = {rf_results["auc"]:.3f})',
            line=dict(color='darkorange', width=2)
        ))
        fig_comp.add_trace(go.Scatter(
            x=[0, 1], y=[0, 1],
            mode='lines',
            name='Random',
            line=dict(color='navy', width=2, dash='dash')
        ))
        
        fig_comp.update_layout(
            title='Comparação de Modelos - Curva ROC',
            xaxis_title='Taxa de Falsos Positivos',
            yaxis_title='Taxa de Verdadeiros Positivos',
            height=500
        )
        fig_comp.show()
    
    # Matriz de Confusão
    cm_xgb = confusion_matrix(y_test, y_pred_xgb)
    fig_cm = px.imshow(
        cm_xgb,
        text_auto=True,
        labels=dict(x="Predito", y="Real", color="Quantidade"),
        x=['Não Notificou', 'Notificou'],
        y=['Não Notificou', 'Notificou'],
        title='Matriz de Confusão - XGBoost',
        color_continuous_scale='Blues'
    )
    fig_cm.show()
    
    # Salvar modelo
    xgb_results = {
        'model': xgb_model,
        'feature_importance': feature_importance_xgb,
        'auc': roc_auc_xgb,
        'predictions': y_pred_xgb,
        'probabilities': y_pred_proba_xgb
    }
    
    print("\n✅ XGBoost treinado com sucesso!")
    print(f"🎯 AUC XGBoost: {roc_auc_xgb:.4f}")
    if rf_results is not None:
        print(f"🎯 AUC Random Forest: {rf_results['auc']:.4f}")
        # CORRIGIDO: usar np.abs ao invés de abs
        diferenca = np.abs(roc_auc_xgb - rf_results['auc'])
        print(f"📊 Diferença: {diferenca:.4f}")
    
else:
    print("⚠️ Dados ML não disponíveis")
    xgb_results = None

In [ ]:
# ============================================================================
# CÉLULA 12: ANÁLISE DE CLUSTERS - K-MEANS (OTIMIZADA)
# ============================================================================

if ml_data is not None:
    
    print("🎯 ANÁLISE DE CLUSTERS - K-MEANS")
    print("=" * 80)
    
    df_ml = ml_data['df_ml']
    
    # Selecionar features para clustering
    cluster_features = [
        'valor_log', 'valor_imposto_infracao', 'valor_multa_infracao',
        'perc_multa', 'regime_encoded', 'cnae_encoded',
        'dias_infracao_ate_nf', 'ano_infracao'
    ]
    
    # Preparar dados
    X_cluster = df_ml[cluster_features].fillna(0)
    
    # OTIMIZAÇÃO: Usar amostra para cálculo de silhouette se dataset muito grande
    if len(X_cluster) > 10000:
        print(f"\n⚡ Dataset grande ({len(X_cluster):,} registros)")
        print("   Usando amostra de 10.000 registros para otimizar cálculo do cotovelo")
        X_cluster_sample = X_cluster.sample(n=10000, random_state=42)
    else:
        X_cluster_sample = X_cluster.copy()
    
    # Normalizar
    scaler_cluster = StandardScaler()
    X_cluster_scaled = scaler_cluster.fit_transform(X_cluster)
    X_cluster_sample_scaled = scaler_cluster.transform(X_cluster_sample)
    
    # Método do Cotovelo para encontrar k ideal
    print("\n📊 Calculando número ideal de clusters (Método do Cotovelo)...")
    
    inertias = []
    silhouette_scores = []
    K_range = range(2, 11)
    
    for k in K_range:
        print(f"   Testando k={k}...", end=" ")
        kmeans_temp = KMeans(n_clusters=k, random_state=42, n_init=10, max_iter=100)
        kmeans_temp.fit(X_cluster_sample_scaled)
        inertias.append(kmeans_temp.inertia_)
        
        # Calcular silhouette na amostra
        sil_score = silhouette_score(X_cluster_sample_scaled, kmeans_temp.labels_, sample_size=5000)
        silhouette_scores.append(sil_score)
        print(f"Silhouette: {sil_score:.3f}")
    
    # Visualizar método do cotovelo
    fig_elbow = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Método do Cotovelo', 'Silhouette Score')
    )
    
    fig_elbow.add_trace(
        go.Scatter(
            x=list(K_range), y=inertias,
            mode='lines+markers',
            name='Inércia',
            line=dict(color='blue', width=2),
            marker=dict(size=10)
        ),
        row=1, col=1
    )
    
    fig_elbow.add_trace(
        go.Scatter(
            x=list(K_range), y=silhouette_scores,
            mode='lines+markers',
            name='Silhouette',
            line=dict(color='red', width=2),
            marker=dict(size=10)
        ),
        row=1, col=2
    )
    
    fig_elbow.update_xaxes(title_text="Número de Clusters", row=1, col=1)
    fig_elbow.update_xaxes(title_text="Número de Clusters", row=1, col=2)
    fig_elbow.update_yaxes(title_text="Inércia", row=1, col=1)
    fig_elbow.update_yaxes(title_text="Silhouette Score", row=1, col=2)
    fig_elbow.update_layout(height=400, showlegend=False, title_text="Análise do Número Ideal de Clusters")
    
    fig_elbow.show()
    
    # Escolher k ideal (maior silhouette)
    k_ideal = K_range[np.argmax(silhouette_scores)]
    max_silhouette = np.max(silhouette_scores)
    print(f"\n🎯 Número ideal de clusters: {k_ideal}")
    print(f"📊 Silhouette Score: {max_silhouette:.4f}")
    
    # Treinar K-Means com k ideal NO DATASET COMPLETO
    print(f"\n🔄 Treinando K-Means com {k_ideal} clusters no dataset completo...")
    
    kmeans = KMeans(n_clusters=k_ideal, random_state=42, n_init=10, max_iter=300)
    df_ml['cluster'] = kmeans.fit_predict(X_cluster_scaled)
    
    # Análise dos clusters
    print("\n📊 CARACTERÍSTICAS DOS CLUSTERS:")
    print("=" * 80)
    
    cluster_analysis = df_ml.groupby('cluster').agg({
        'cnpj': 'count',
        'valor_total_infracao': ['mean', 'sum', 'std'],
        'valor_total_nf': ['mean', 'sum'],
        'gerou_notificacao': ['mean', 'sum'],
        'ciclo_completo': 'mean',
        'dias_infracao_ate_nf': 'mean',
        'pagou_dp_infracao': 'mean'
    }).round(2)
    
    print(cluster_analysis)
    
    # Distribuição dos clusters
    cluster_counts = df_ml['cluster'].value_counts().sort_index()
    
    fig_dist = go.Figure(data=[
        go.Bar(
            x=cluster_counts.index,
            y=cluster_counts.values,
            text=cluster_counts.values,
            textposition='auto',
            marker_color='lightseagreen'
        )
    ])
    
    fig_dist.update_layout(
        title='Distribuição de Fiscalizações por Cluster',
        xaxis_title='Cluster',
        yaxis_title='Quantidade de Fiscalizações',
        height=400
    )
    
    fig_dist.show()
    
    # PCA para visualização 2D
    print("\n🔄 Aplicando PCA para visualização 2D...")
    
    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X_cluster_scaled)
    
    df_ml['pca1'] = X_pca[:, 0]
    df_ml['pca2'] = X_pca[:, 1]
    
    print(f"📊 Variância explicada: {pca.explained_variance_ratio_.sum()*100:.2f}%")
    
    # Visualização dos clusters no espaço PCA
    n_sample = int(np.minimum(5000, len(df_ml)))
    df_sample = df_ml.sample(n=n_sample, random_state=42)
    
    fig_pca = px.scatter(
        df_sample,
        x='pca1',
        y='pca2',
        color='cluster',
        hover_data=['valor_total_infracao', 'regime_tributario', 'cnae_secao_descricao'],
        title='Visualização dos Clusters no Espaço PCA',
        labels={'pca1': f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}%)',
                'pca2': f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}%)'},
        color_continuous_scale='Viridis'
    )
    
    fig_pca.update_traces(marker=dict(size=5, opacity=0.6))
    fig_pca.update_layout(height=600)
    fig_pca.show()
    
    # Perfil de cada cluster
    print("\n🎯 PERFIL DOS CLUSTERS:")
    print("=" * 80)
    
    for cluster_id in sorted(df_ml['cluster'].unique()):
        cluster_data = df_ml[df_ml['cluster'] == cluster_id]
        
        print(f"\n🔹 CLUSTER {cluster_id} ({len(cluster_data):,} fiscalizações)")
        print(f"  Valor médio infração: {format_currency(cluster_data['valor_total_infracao'].mean())}")
        print(f"  Taxa notificação: {cluster_data['gerou_notificacao'].mean()*100:.2f}%")
        print(f"  Taxa ciclo completo: {cluster_data['ciclo_completo'].mean()*100:.2f}%")
        print(f"  Média dias até NF: {cluster_data['dias_infracao_ate_nf'].mean():.0f} dias")
        
        # Top 3 setores
        top_setores = cluster_data['cnae_secao_descricao'].value_counts().head(3)
        print(f"  Top 3 setores:")
        for setor, qtd in top_setores.items():
            print(f"    - {setor}: {qtd} ({qtd/len(cluster_data)*100:.1f}%)")
    
    # Salvar resultados
    cluster_results = {
        'model': kmeans,
        'scaler': scaler_cluster,
        'pca': pca,
        'k_ideal': k_ideal,
        'silhouette': max_silhouette,
        'df_clustered': df_ml
    }
    
    print("\n✅ Análise de clusters concluída!")
    
else:
    print("⚠️ Dados ML não disponíveis")
    cluster_results = None

In [ ]:
# ============================================================================
# CÉLULA 13: ANÁLISE DE DEFESA PRÉVIA E PAGAMENTOS
# ============================================================================

if not df_fisc.empty:
    
    print("💰 ANÁLISE DE DEFESA PRÉVIA E PAGAMENTOS")
    print("=" * 80)
    
    # Análise de pagamentos em DP
    total_fiscalizacoes = len(df_fisc)
    pagou_dp = df_fisc['pagou_dp_infracao'].sum()
    
    print(f"\n📊 ESTATÍSTICAS DE DEFESA PRÉVIA:")
    print(f"  Total de fiscalizações: {total_fiscalizacoes:,}")
    print(f"  Pagamentos em DP: {pagou_dp:,}")
    print(f"  Taxa de pagamento DP: {pagou_dp/total_fiscalizacoes*100:.2f}%")
    
    # Análise por ano
    dp_por_ano = df_fisc.groupby('ano_infracao').agg({
        'cnpj': 'count',
        'pagou_dp_infracao': 'sum',
        'valor_total_infracao': 'sum',
        'gerou_notificacao': 'sum'
    }).reset_index()
    
    dp_por_ano.columns = ['ano', 'total_fisc', 'pagamentos_dp', 'valor_total', 'notificacoes']
    dp_por_ano['taxa_dp'] = dp_por_ano['pagamentos_dp'] / dp_por_ano['total_fisc'] * 100
    dp_por_ano['taxa_notif'] = dp_por_ano['notificacoes'] / dp_por_ano['total_fisc'] * 100
    
    print("\n📊 Evolução de Pagamentos DP por Ano:")
    print(dp_por_ano)
    
    # Visualização - Evolução DP
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=('Quantidade de Pagamentos em Defesa Prévia', 'Taxa de Pagamento DP vs Taxa de Notificação'),
        specs=[[{"secondary_y": False}], [{"secondary_y": False}]]
    )
    
    # Gráfico 1: Quantidade
    fig.add_trace(
        go.Bar(
            x=dp_por_ano['ano'],
            y=dp_por_ano['pagamentos_dp'],
            name='Pagamentos DP',
            marker_color='lightcoral',
            text=dp_por_ano['pagamentos_dp'],
            textposition='auto'
        ),
        row=1, col=1
    )
    
    # Gráfico 2: Taxas
    fig.add_trace(
        go.Scatter(
            x=dp_por_ano['ano'],
            y=dp_por_ano['taxa_dp'],
            name='Taxa Pagamento DP',
            mode='lines+markers',
            line=dict(color='red', width=3)
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=dp_por_ano['ano'],
            y=dp_por_ano['taxa_notif'],
            name='Taxa Notificação',
            mode='lines+markers',
            line=dict(color='blue', width=3)
        ),
        row=2, col=1
    )
    
    fig.update_xaxes(title_text="Ano", row=2, col=1)
    fig.update_yaxes(title_text="Quantidade", row=1, col=1)
    fig.update_yaxes(title_text="Taxa (%)", row=2, col=1)
    
    fig.update_layout(
        height=800,
        showlegend=True,
        title_text="Análise de Defesa Prévia ao Longo do Tempo"
    )
    
    fig.show()
    
    # Análise: Pagamento DP vs Valor
    print("\n💵 ANÁLISE POR FAIXA DE VALOR:")
    print("=" * 80)
    
    df_fisc['faixa_valor'] = pd.cut(
        df_fisc['valor_total_infracao'],
        bins=[0, 50000, 100000, 500000, 1000000, np.inf],
        labels=['< 50k', '50k-100k', '100k-500k', '500k-1M', '> 1M']
    )
    
    dp_por_valor = df_fisc.groupby('faixa_valor').agg({
        'cnpj': 'count',
        'pagou_dp_infracao': 'sum',
        'valor_total_infracao': ['mean', 'sum'],
        'gerou_notificacao': 'sum'
    }).reset_index()
    
    dp_por_valor.columns = ['faixa_valor', 'total', 'pagamentos_dp', 'valor_medio', 'valor_total', 'notificacoes']
    dp_por_valor['taxa_dp'] = dp_por_valor['pagamentos_dp'] / dp_por_valor['total'] * 100
    dp_por_valor['taxa_notif'] = dp_por_valor['notificacoes'] / dp_por_valor['total'] * 100
    
    print(dp_por_valor)
    
    # Visualização por faixa de valor
    fig2 = go.Figure()
    
    fig2.add_trace(go.Bar(
        x=dp_por_valor['faixa_valor'],
        y=dp_por_valor['taxa_dp'],
        name='Taxa Pagamento DP',
        marker_color='salmon',
        text=dp_por_valor['taxa_dp'].round(2),
        textposition='auto'
    ))
    
    fig2.add_trace(go.Bar(
        x=dp_por_valor['faixa_valor'],
        y=dp_por_valor['taxa_notif'],
        name='Taxa Notificação',
        marker_color='steelblue',
        text=dp_por_valor['taxa_notif'].round(2),
        textposition='auto'
    ))
    
    fig2.update_layout(
        title='Taxa de Pagamento DP vs Notificação por Faixa de Valor',
        xaxis_title='Faixa de Valor da Infração',
        yaxis_title='Taxa (%)',
        barmode='group',
        height=500
    )
    
    fig2.show()
    
    # Análise por regime tributário
    print("\n🏛️ ANÁLISE POR REGIME TRIBUTÁRIO:")
    print("=" * 80)
    
    dp_por_regime = df_fisc.groupby('regime_tributario').agg({
        'cnpj': 'count',
        'pagou_dp_infracao': 'sum',
        'gerou_notificacao': 'sum',
        'valor_total_infracao': 'sum'
    }).reset_index()
    
    dp_por_regime.columns = ['regime', 'total', 'pagamentos_dp', 'notificacoes', 'valor_total']
    dp_por_regime['taxa_dp'] = dp_por_regime['pagamentos_dp'] / dp_por_regime['total'] * 100
    dp_por_regime['taxa_notif'] = dp_por_regime['notificacoes'] / dp_por_regime['total'] * 100
    dp_por_regime = dp_por_regime.sort_values('total', ascending=False)
    
    print(dp_por_regime.head(10))
    
    # Visualização por regime
    top_regimes = dp_por_regime.head(8)
    
    fig3 = px.bar(
        top_regimes,
        x='regime',
        y=['taxa_dp', 'taxa_notif'],
        title='Comparação Taxa DP vs Notificação por Regime Tributário (Top 8)',
        labels={'value': 'Taxa (%)', 'variable': 'Métrica'},
        barmode='group',
        color_discrete_map={'taxa_dp': 'coral', 'taxa_notif': 'teal'}
    )
    
    fig3.update_layout(height=500, xaxis_tickangle=-45)
    fig3.show()
    
    print("\n✅ Análise de Defesa Prévia concluída!")

else:
    print("⚠️ DataFrame vazio - pulando análise de DP")

In [ ]:
# ============================================================================
# CÉLULA 14: ANÁLISE DE EFETIVIDADE - SCORES (COMPLETA)
# ============================================================================

query_scores = """
SELECT 
    cnpj,
    nm_razao_social,
    gerfe,
    municipio,
    cnae_secao_descricao,
    regime_tributario,
    ano_infracao,
    CAST(COALESCE(valor_total_infracao, 0) AS DOUBLE) AS valor_total_infracao,
    CAST(COALESCE(valor_total_nf, 0) AS DOUBLE) AS valor_total_nf,
    CAST(COALESCE(gerou_notificacao, 0) AS INT) AS gerou_notificacao,
    CAST(COALESCE(ciclo_completo, 0) AS INT) AS ciclo_completo,
    CAST(COALESCE(dias_infracao_ate_nf, 0) AS DOUBLE) AS dias_infracao_ate_nf,
    CAST(COALESCE(score_efetividade_final, 0) AS DOUBLE) AS score_efetividade_final,
    classificacao_efetividade,
    CAST(COALESCE(score_geracao_nf, 0) AS DOUBLE) AS score_geracao_nf,
    CAST(COALESCE(score_ciclo, 0) AS DOUBLE) AS score_ciclo,
    CAST(COALESCE(score_valor_notificado, 0) AS DOUBLE) AS score_valor_notificado,
    CAST(COALESCE(score_tempestividade, 0) AS DOUBLE) AS score_tempestividade
FROM teste.fisca_scores_efetividade
WHERE ano_infracao >= 2020
"""

df_scores = load_spark_to_pandas_safe(
    query_scores,
    limit=150000,
    view_name="vw_scores"
)

if not df_scores.empty:
    
    print("📊 ANÁLISE DE SCORES DE EFETIVIDADE")
    print("=" * 80)
    
    # Estatísticas dos scores
    print("\n📈 ESTATÍSTICAS DOS SCORES:")
    print(f"  Score Médio Geral: {df_scores['score_efetividade_final'].mean():.2f}")
    print(f"  Score Mediano: {df_scores['score_efetividade_final'].median():.2f}")
    print(f"  Desvio Padrão: {df_scores['score_efetividade_final'].std():.2f}")
    print(f"  Score Mínimo: {df_scores['score_efetividade_final'].min():.2f}")
    print(f"  Score Máximo: {df_scores['score_efetividade_final'].max():.2f}")
    
    # Distribuição por classificação
    print("\n🎯 DISTRIBUIÇÃO POR CLASSIFICAÇÃO:")
    class_dist = df_scores['classificacao_efetividade'].value_counts()
    print(class_dist)
    
    # Visualização - Distribuição de Scores
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Distribuição do Score Final',
            'Distribuição por Classificação',
            'Scores por Componente',
            'Evolução do Score ao Longo do Tempo'
        ),
        specs=[
            [{"type": "histogram"}, {"type": "bar"}],
            [{"type": "box"}, {"type": "scatter"}]
        ]
    )
    
    # 1. Histograma do Score Final
    fig.add_trace(
        go.Histogram(
            x=df_scores['score_efetividade_final'],
            nbinsx=50,
            name='Score Final',
            marker_color='steelblue'
        ),
        row=1, col=1
    )
    
    # 2. Distribuição por Classificação
    fig.add_trace(
        go.Bar(
            x=class_dist.index,
            y=class_dist.values,
            name='Classificação',
            marker_color='coral',
            text=class_dist.values,
            textposition='auto'
        ),
        row=1, col=2
    )
    
    # 3. Box plot dos componentes
    score_components = ['score_geracao_nf', 'score_ciclo', 'score_valor_notificado', 'score_tempestividade']
    for comp in score_components:
        fig.add_trace(
            go.Box(
                y=df_scores[comp],
                name=comp.replace('score_', '').replace('_', ' ').title(),
                boxmean='sd'
            ),
            row=2, col=1
        )
    
    # 4. Evolução temporal
    score_por_ano = df_scores.groupby('ano_infracao')['score_efetividade_final'].mean().reset_index()
    
    fig.add_trace(
        go.Scatter(
            x=score_por_ano['ano_infracao'],
            y=score_por_ano['score_efetividade_final'],
            mode='lines+markers',
            name='Score Médio',
            line=dict(color='green', width=3),
            marker=dict(size=10)
        ),
        row=2, col=2
    )
    
    fig.update_layout(
        height=900,
        showlegend=True,
        title_text="📊 Análise Completa de Scores de Efetividade"
    )
    
    fig.update_xaxes(title_text="Score", row=1, col=1)
    fig.update_xaxes(title_text="Classificação", row=1, col=2)
    fig.update_xaxes(title_text="Ano", row=2, col=2)
    fig.update_yaxes(title_text="Frequência", row=1, col=1)
    fig.update_yaxes(title_text="Quantidade", row=1, col=2)
    fig.update_yaxes(title_text="Score", row=2, col=1)
    fig.update_yaxes(title_text="Score Médio", row=2, col=2)
    
    fig.show()
    
    # Análise por Gerência
    print("\n🏢 TOP 10 GERÊNCIAS POR SCORE:")
    print("=" * 80)
    
    score_por_gerencia = df_scores.groupby('gerfe').agg({
        'cnpj': 'count',
        'score_efetividade_final': 'mean',
        'valor_total_nf': 'sum',
        'gerou_notificacao': 'sum'
    }).reset_index()
    
    score_por_gerencia.columns = ['gerencia', 'qtd_fisc', 'score_medio', 'valor_total_nf', 'total_nfs']
    score_por_gerencia = score_por_gerencia.sort_values('score_medio', ascending=False)
    
    print(score_por_gerencia.head(10))
    
    # Visualização Gerências
    top_gerencias = score_por_gerencia.head(15)
    
    fig2 = px.bar(
        top_gerencias,
        x='gerencia',
        y='score_medio',
        title='Top 15 Gerências por Score de Efetividade',
        labels={'score_medio': 'Score Médio', 'gerencia': 'Gerência'},
        color='score_medio',
        color_continuous_scale='RdYlGn',
        text='score_medio'
    )
    
    fig2.update_traces(texttemplate='%{text:.1f}', textposition='outside')
    fig2.update_layout(height=600, xaxis_tickangle=-45)
    fig2.show()
    
    # Análise por Setor
    print("\n🏭 TOP 10 SETORES POR SCORE:")
    print("=" * 80)
    
    score_por_setor = df_scores.groupby('cnae_secao_descricao').agg({
        'cnpj': 'count',
        'score_efetividade_final': 'mean',
        'valor_total_nf': 'sum'
    }).reset_index()
    
    score_por_setor.columns = ['setor', 'qtd_fisc', 'score_medio', 'valor_total_nf']
    score_por_setor = score_por_setor[score_por_setor['qtd_fisc'] >= 20]  # Mínimo 20 fiscalizações
    score_por_setor = score_por_setor.sort_values('score_medio', ascending=False)
    
    print(score_por_setor.head(10))
    
    # Scatter plot: Score vs Valor - CORRIGIDO
    n_sample_scores = int(np.minimum(5000, len(df_scores)))
    df_scores_sample = df_scores.sample(n=n_sample_scores, random_state=42)
    
    fig3 = px.scatter(
        df_scores_sample,
        x='valor_total_infracao',
        y='score_efetividade_final',
        color='classificacao_efetividade',
        size='valor_total_nf',
        hover_data=['nm_razao_social', 'municipio'],
        title='Relação entre Valor da Infração e Score de Efetividade',
        labels={
            'valor_total_infracao': 'Valor Total Infração (R$)',
            'score_efetividade_final': 'Score de Efetividade'
        },
        color_discrete_map={
            'MUITO EFETIVA': 'green',
            'EFETIVA': 'lightgreen',
            'MODERADAMENTE EFETIVA': 'yellow',
            'POUCO EFETIVA': 'red'
        }
    )
    
    fig3.update_layout(height=600)
    fig3.update_xaxes(type="log")
    fig3.show()
    
    print("\n✅ Análise de efetividade concluída!")

else:
    print("⚠️ DataFrame vazio - pulando análise de scores")

In [ ]:
# ============================================================================
# CÉLULA 15: ANÁLISE DE PRODUTIVIDADE POR AFRE
# ============================================================================

query_afres = """
SELECT 
    matricula_afre,
    nome_afre,
    ano,
    CAST(COALESCE(meses_ativos, 0) AS DOUBLE) AS meses_ativos,
    CAST(COALESCE(dias_ativos, 0) AS DOUBLE) AS dias_ativos,
    CAST(COALESCE(qtd_infracoes, 0) AS DOUBLE) AS qtd_infracoes,
    CAST(COALESCE(qtd_empresas_fiscalizadas, 0) AS DOUBLE) AS qtd_empresas_fiscalizadas,
    CAST(COALESCE(qtd_infracoes_com_ciencia, 0) AS DOUBLE) AS qtd_infracoes_com_ciencia,
    CAST(COALESCE(valor_total_infracoes, 0) AS DOUBLE) AS valor_total_infracoes,
    CAST(COALESCE(qtd_nfs, 0) AS DOUBLE) AS qtd_nfs,
    CAST(COALESCE(valor_total_lancado, 0) AS DOUBLE) AS valor_total_lancado,
    CAST(COALESCE(infracoes_por_mes, 0) AS DOUBLE) AS infracoes_por_mes,
    CAST(COALESCE(nfs_por_mes, 0) AS DOUBLE) AS nfs_por_mes,
    CAST(COALESCE(taxa_conversao_infracao_nf, 0) AS DOUBLE) AS taxa_conversao_infracao_nf,
    CAST(COALESCE(valor_medio_mensal, 0) AS DOUBLE) AS valor_medio_mensal
FROM teste.fisca_metricas_por_afre
WHERE meses_ativos >= 6
  AND ano >= 2020
"""

df_afres = load_spark_to_pandas_safe(
    query_afres,
    limit=100000,
    view_name="vw_afres"
)

if not df_afres.empty:
    
    print("👥 ANÁLISE DE PRODUTIVIDADE POR AFRE")
    print("=" * 80)
    
    # Ano mais recente
    ano_recente = df_afres['ano'].max()
    df_afres_recente = df_afres[df_afres['ano'] == ano_recente]
    
    print(f"\n📅 Análise do ano: {ano_recente}")
    print(f"  Total de AFREs ativos: {df_afres_recente['matricula_afre'].nunique():,}")
    print(f"  Total de infrações: {df_afres_recente['qtd_infracoes'].sum():,.0f}")
    print(f"  Total de NFs: {df_afres_recente['qtd_nfs'].sum():,.0f}")
    print(f"  Valor total lançado: {format_currency(df_afres_recente['valor_total_lancado'].sum())}")
    
    # Estatísticas de produtividade
    print(f"\n📊 ESTATÍSTICAS DE PRODUTIVIDADE ({ano_recente}):")
    print(f"  Infrações/mês - Média: {df_afres_recente['infracoes_por_mes'].mean():.2f}")
    print(f"  Infrações/mês - Mediana: {df_afres_recente['infracoes_por_mes'].median():.2f}")
    print(f"  NFs/mês - Média: {df_afres_recente['nfs_por_mes'].mean():.2f}")
    print(f"  NFs/mês - Mediana: {df_afres_recente['nfs_por_mes'].median():.2f}")
    print(f"  Taxa conversão - Média: {df_afres_recente['taxa_conversao_infracao_nf'].mean():.2f}%")
    
    # Top 20 AFREs mais produtivos
    top_afres = df_afres_recente.nlargest(20, 'valor_total_lancado')
    
    print(f"\n🏆 TOP 20 AFREs POR VALOR LANÇADO ({ano_recente}):")
    print(top_afres[['nome_afre', 'qtd_nfs', 'valor_total_lancado', 'nfs_por_mes', 'taxa_conversao_infracao_nf']])
    
    # Visualização - Top 20 AFREs
    fig = make_subplots(
        rows=2, cols=1,
        subplot_titles=(
            f'Top 20 AFREs por Valor Lançado ({ano_recente})',
            f'Top 20 AFREs por Produtividade (NFs/mês) ({ano_recente})'
        )
    )
    
    # Gráfico 1: Valor lançado
    fig.add_trace(
        go.Bar(
            x=top_afres['nome_afre'],
            y=top_afres['valor_total_lancado'] / 1_000_000,
            name='Valor Lançado',
            marker_color='mediumseagreen',
            text=(top_afres['valor_total_lancado'] / 1_000_000).round(2),
            textposition='auto'
        ),
        row=1, col=1
    )
    
    # Gráfico 2: NFs por mês
    top_afres_prod = df_afres_recente.nlargest(20, 'nfs_por_mes')
    
    fig.add_trace(
        go.Bar(
            x=top_afres_prod['nome_afre'],
            y=top_afres_prod['nfs_por_mes'],
            name='NFs/mês',
            marker_color='lightsalmon',
            text=top_afres_prod['nfs_por_mes'].round(2),
            textposition='auto'
        ),
        row=2, col=1
    )
    
    fig.update_xaxes(tickangle=-45, row=1, col=1)
    fig.update_xaxes(tickangle=-45, row=2, col=1)
    fig.update_yaxes(title_text="R$ Milhões", row=1, col=1)
    fig.update_yaxes(title_text="NFs/mês", row=2, col=1)
    
    fig.update_layout(height=900, showlegend=False, title_text="📊 Produtividade dos AFREs")
    fig.show()
    
    # Distribuição de produtividade
    print(f"\n📊 DISTRIBUIÇÃO DE PRODUTIVIDADE ({ano_recente}):")
    print("=" * 80)
    
    df_afres_recente['faixa_produtividade'] = pd.cut(
        df_afres_recente['nfs_por_mes'],
        bins=[0, 0.5, 1, 2, 3, np.inf],
        labels=['Muito Baixa (<0.5)', 'Baixa (0.5-1)', 'Média (1-2)', 'Alta (2-3)', 'Muito Alta (>3)']
    )
    
    dist_prod = df_afres_recente['faixa_produtividade'].value_counts().sort_index()
    print(dist_prod)
    
    # Visualização distribuição
    fig2 = go.Figure(data=[
        go.Pie(
            labels=dist_prod.index,
            values=dist_prod.values,
            hole=0.4,
            marker=dict(colors=['red', 'orange', 'yellow', 'lightgreen', 'green'])
        )
    ])
    
    fig2.update_layout(
        title=f'Distribuição de AFREs por Faixa de Produtividade ({ano_recente})',
        height=500
    )
    fig2.show()
    
    # Evolução temporal da produtividade
    print("\n📈 EVOLUÇÃO TEMPORAL DA PRODUTIVIDADE:")
    print("=" * 80)
    
    prod_temporal = df_afres.groupby('ano').agg({
        'matricula_afre': 'count',
        'qtd_infracoes': 'sum',
        'qtd_nfs': 'sum',
        'valor_total_lancado': 'sum',
        'infracoes_por_mes': 'mean',
        'nfs_por_mes': 'mean',
        'taxa_conversao_infracao_nf': 'mean'
    }).reset_index()
    
    prod_temporal.columns = [
        'ano', 'qtd_afres', 'total_infracoes', 'total_nfs',
        'valor_total', 'media_infracoes_mes', 'media_nfs_mes', 'taxa_conversao'
    ]
    
    print(prod_temporal)
    
    # Visualização evolução
    fig3 = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Evolução Quantidade de AFREs',
            'Evolução Total de NFs',
            'Evolução Produtividade Média (NFs/mês)',
            'Evolução Taxa de Conversão Média'
        )
    )
    
    fig3.add_trace(
        go.Scatter(x=prod_temporal['ano'], y=prod_temporal['qtd_afres'],
                   mode='lines+markers', name='AFREs', line=dict(color='blue', width=3)),
        row=1, col=1
    )
    
    fig3.add_trace(
        go.Scatter(x=prod_temporal['ano'], y=prod_temporal['total_nfs'],
                   mode='lines+markers', name='Total NFs', line=dict(color='green', width=3)),
        row=1, col=2
    )
    
    fig3.add_trace(
        go.Scatter(x=prod_temporal['ano'], y=prod_temporal['media_nfs_mes'],
                   mode='lines+markers', name='NFs/mês', line=dict(color='orange', width=3)),
        row=2, col=1
    )
    
    fig3.add_trace(
        go.Scatter(x=prod_temporal['ano'], y=prod_temporal['taxa_conversao'],
                   mode='lines+markers', name='Taxa Conversão', line=dict(color='red', width=3)),
        row=2, col=2
    )
    
    fig3.update_layout(height=800, showlegend=False, title_text="📊 Evolução Temporal da Produtividade")
    fig3.show()
    
    print("\n✅ Análise de produtividade por AFRE concluída!")

else:
    print("⚠️ DataFrame vazio - pulando análise de AFREs")

In [ ]:
# ============================================================================
# CÉLULA 16: ANÁLISE DO FUNIL DE FISCALIZAÇÃO
# ============================================================================
# Baseado na Portaria SEF 031/2021 e Ato DIAT 004/2021
# Monitoramento → Acompanhamento → PAF (Constituição do Crédito)

if not df_fisc.empty:
    
    print("🔍 ANÁLISE DO FUNIL DE FISCALIZAÇÃO")
    print("=" * 80)
    print("Baseado na Portaria SEF 031/2021:")
    print("  I - Monitoramento (observação permanente)")
    print("  II - Acompanhamento (inconsistências detectadas)")
    print("  III - PAF (Constituição do Crédito Tributário)")
    print("=" * 80)
    
    # Carregar dados de acompanhamentos
    query_acomp = """
    SELECT 
        ano_os,
        COUNT(DISTINCT id_acompanhamento) AS qtd_acompanhamentos,
        COUNT(DISTINCT cnpj) AS empresas_acompanhadas,
        SUM(CASE WHEN gerou_documento = 1 THEN 1 ELSE 0 END) AS acomp_com_documento,
        SUM(CASE WHEN foi_encerrado = 1 THEN 1 ELSE 0 END) AS acomp_encerrados,
        AVG(CASE WHEN dias_duracao_acao > 0 THEN dias_duracao_acao END) AS media_dias_duracao
    FROM teste.fisca_acompanhamentos
    WHERE ano_os >= 2020
    GROUP BY ano_os
    ORDER BY ano_os DESC
    """
    
    df_acomp_resumo = load_spark_to_pandas_safe(
        query_acomp,
        view_name="vw_acomp_resumo"
    )
    
    if not df_acomp_resumo.empty:
        print("\n📊 RESUMO DE ACOMPANHAMENTOS POR ANO:")
        print(df_acomp_resumo)
        
        # Calcular conversão Acompanhamento → PAF
        funil_por_ano = pd.merge(
            df_acomp_resumo[['ano_os', 'qtd_acompanhamentos', 'empresas_acompanhadas']],
            df_dashboard[['ano', 'qtd_infracoes_lavradas', 'empresas_fiscalizadas']],
            left_on='ano_os',
            right_on='ano',
            how='outer'
        ).fillna(0)
        
        funil_por_ano['taxa_conversao_acomp_paf'] = np.where(
            funil_por_ano['qtd_acompanhamentos'] > 0,
            funil_por_ano['qtd_infracoes_lavradas'] / funil_por_ano['qtd_acompanhamentos'] * 100,
            0
        )
        
        print("\n📊 FUNIL: ACOMPANHAMENTO → PAF (CONSTITUIÇÃO DO CRÉDITO):")
        print("=" * 80)
        print(funil_por_ano[['ano', 'qtd_acompanhamentos', 'qtd_infracoes_lavradas', 'taxa_conversao_acomp_paf']])
        
        # Visualização do Funil
        fig = go.Figure()
        
        anos = funil_por_ano['ano'].astype(int)
        
        # Funil invertido
        fig.add_trace(go.Funnel(
            name='2020',
            y=['Acompanhamentos', 'PAF (Infrações)', 'Notificações Fiscais', 'Ciclos Completos'],
            x=[
                df_acomp_resumo[df_acomp_resumo['ano_os'] == 2020]['qtd_acompanhamentos'].sum(),
                df_dashboard[df_dashboard['ano'] == 2020]['qtd_infracoes_lavradas'].sum(),
                df_dashboard[df_dashboard['ano'] == 2020]['qtd_nfs_emitidas'].sum(),
                df_dashboard[df_dashboard['ano'] == 2020]['qtd_ciclos_completos'].sum()
            ],
            textinfo="value+percent initial"
        ))
        
        fig.update_layout(
            title='Funil de Fiscalização 2020: Acompanhamento → PAF → NF → Ciclo Completo',
            height=500
        )
        fig.show()
        
        # Evolução temporal do funil
        fig2 = make_subplots(
            rows=2, cols=1,
            subplot_titles=(
                'Evolução: Acompanhamentos vs PAF (Infrações)',
                'Taxa de Conversão Acompanhamento → PAF'
            )
        )
        
        fig2.add_trace(
            go.Bar(
                x=funil_por_ano['ano'],
                y=funil_por_ano['qtd_acompanhamentos'],
                name='Acompanhamentos',
                marker_color='lightblue'
            ),
            row=1, col=1
        )
        
        fig2.add_trace(
            go.Bar(
                x=funil_por_ano['ano'],
                y=funil_por_ano['qtd_infracoes_lavradas'],
                name='PAF (Infrações)',
                marker_color='indianred'
            ),
            row=1, col=1
        )
        
        fig2.add_trace(
            go.Scatter(
                x=funil_por_ano['ano'],
                y=funil_por_ano['taxa_conversao_acomp_paf'],
                name='Taxa Conversão',
                mode='lines+markers',
                line=dict(color='green', width=3),
                marker=dict(size=10)
            ),
            row=2, col=1
        )
        
        fig2.update_xaxes(title_text="Ano", row=2, col=1)
        fig2.update_yaxes(title_text="Quantidade", row=1, col=1)
        fig2.update_yaxes(title_text="Taxa (%)", row=2, col=1)
        
        fig2.update_layout(height=800, title_text="Análise do Funil de Fiscalização")
        fig2.show()
    
    # Análise de efetividade do acompanhamento
    print("\n🎯 EFETIVIDADE DO ACOMPANHAMENTO:")
    print("=" * 80)
    print("Conforme Portaria SEF 031/2021, Art. 6º, § 2º:")
    print("'As ações fiscais auxiliares de acompanhamento podem ser utilizadas para")
    print("propor ao sujeito passivo que, espontaneamente, regularize ou preste")
    print("esclarecimento sobre inconsistências detectadas.'")
    print("=" * 80)
    
    # Calcular empresas que regularizaram espontaneamente
    # (empresas com acompanhamento mas SEM PAF constituído)
    query_regularizacao = """
    SELECT 
        a.ano_os,
        COUNT(DISTINCT a.cnpj) AS empresas_acompanhadas,
        COUNT(DISTINCT CASE WHEN f.cnpj IS NULL THEN a.cnpj END) AS empresas_sem_paf,
        COUNT(DISTINCT CASE WHEN f.cnpj IS NOT NULL THEN a.cnpj END) AS empresas_com_paf
    FROM teste.fisca_acompanhamentos a
    LEFT JOIN teste.fisca_fiscalizacoes_consolidadas f
        ON a.cnpj = f.cnpj
        AND a.ano_os = f.ano_infracao
    WHERE a.ano_os >= 2020
        AND a.cnpj IS NOT NULL
    GROUP BY a.ano_os
    ORDER BY a.ano_os DESC
    """
    
    df_reg = load_spark_to_pandas_safe(query_regularizacao, view_name="vw_regularizacao")
    
    if not df_reg.empty:
        df_reg['taxa_regularizacao_espontanea'] = (
            df_reg['empresas_sem_paf'] / df_reg['empresas_acompanhadas'] * 100
        )
        
        print("\n📊 ESTIMATIVA DE REGULARIZAÇÃO ESPONTÂNEA:")
        print("(Empresas com acompanhamento que NÃO geraram PAF)")
        print(df_reg)
        
        # Visualização
        fig3 = px.bar(
            df_reg,
            x='ano_os',
            y=['empresas_sem_paf', 'empresas_com_paf'],
            title='Regularização Espontânea vs Constituição de PAF',
            labels={'value': 'Quantidade de Empresas', 'ano_os': 'Ano'},
            barmode='stack',
            color_discrete_map={
                'empresas_sem_paf': 'lightgreen',
                'empresas_com_paf': 'salmon'
            }
        )
        
        fig3.update_layout(
            height=500,
            legend_title_text='Situação',
            yaxis_title="Empresas"
        )
        fig3.show()
        
        print("\n💡 INSIGHT:")
        print(f"Taxa média de regularização espontânea: {df_reg['taxa_regularizacao_espontanea'].mean():.2f}%")
        print("Empresas que receberam acompanhamento e regularizaram SEM necessidade de PAF")
    
    print("\n✅ Análise do funil de fiscalização concluída!")

else:
    print("⚠️ Dados insuficientes para análise do funil")

In [ ]:
# ============================================================================
# CÉLULA 17: ANÁLISE DE TEMPESTIVIDADE E SLA (Service Level Agreement)
# ============================================================================

if not df_fisc.empty:
    
    print("⏱️ ANÁLISE DE TEMPESTIVIDADE DAS AÇÕES FISCAIS")
    print("=" * 80)
    print("Baseado no Ato DIAT 004/2021, Art. 3º, § 1º, III:")
    print("'O termo de instauração do PAF conterá o prazo de execução da ação fiscal'")
    print("=" * 80)
    
    # Análise de tempos
    tempos_analysis = df_fisc[df_fisc['dias_infracao_ate_nf'] > 0].copy()
    
    print(f"\n📊 ESTATÍSTICAS DE TEMPO (Infração → Notificação Fiscal):")
    print(f"  Total de casos com NF: {len(tempos_analysis):,}")
    print(f"  Média: {tempos_analysis['dias_infracao_ate_nf'].mean():.0f} dias")
    print(f"  Mediana: {tempos_analysis['dias_infracao_ate_nf'].median():.0f} dias")
    print(f"  Desvio Padrão: {tempos_analysis['dias_infracao_ate_nf'].std():.0f} dias")
    print(f"  Mínimo: {tempos_analysis['dias_infracao_ate_nf'].min():.0f} dias")
    print(f"  Máximo: {tempos_analysis['dias_infracao_ate_nf'].max():.0f} dias")
    
    # Percentis
    print(f"\n📊 DISTRIBUIÇÃO POR PERCENTIS:")
    percentiles = [10, 25, 50, 75, 90, 95, 99]
    for p in percentiles:
        valor = np.percentile(tempos_analysis['dias_infracao_ate_nf'], p)
        print(f"  P{p}: {valor:.0f} dias")
    
    # Definir faixas de SLA
    tempos_analysis['faixa_sla'] = pd.cut(
        tempos_analysis['dias_infracao_ate_nf'],
        bins=[0, 30, 60, 90, 180, 365, np.inf],
        labels=['≤ 30 dias (Excelente)', '31-60 dias (Ótimo)', 
                '61-90 dias (Bom)', '91-180 dias (Regular)',
                '181-365 dias (Lento)', '> 365 dias (Crítico)']
    )
    
    sla_dist = tempos_analysis['faixa_sla'].value_counts().sort_index()
    sla_pct = (sla_dist / len(tempos_analysis) * 100).round(2)
    
    print("\n📊 DISTRIBUIÇÃO POR FAIXA DE SLA:")
    for faixa, qtd, pct in zip(sla_dist.index, sla_dist.values, sla_pct.values):
        print(f"  {faixa}: {qtd:,} ({pct}%)")
    
    # Visualização - Distribuição de Tempos
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Histograma - Dias até Notificação',
            'Distribuição por Faixa de SLA',
            'Box Plot por Ano',
            'Evolução da Média ao Longo do Tempo'
        ),
        specs=[
            [{"type": "histogram"}, {"type": "bar"}],
            [{"type": "box"}, {"type": "scatter"}]
        ]
    )
    
    # 1. Histograma
    fig.add_trace(
        go.Histogram(
            x=tempos_analysis['dias_infracao_ate_nf'],
            nbinsx=100,
            name='Dias até NF',
            marker_color='steelblue'
        ),
        row=1, col=1
    )
    
    # 2. Distribuição SLA
    fig.add_trace(
        go.Bar(
            x=sla_dist.values,
            y=sla_dist.index.astype(str),
            orientation='h',
            text=sla_pct.values,
            texttemplate='%{text}%',
            textposition='auto',
            marker_color=['green', 'lightgreen', 'yellow', 'orange', 'red', 'darkred']
        ),
        row=1, col=2
    )
    
    # 3. Box plot por ano
    for ano in sorted(tempos_analysis['ano_infracao'].unique()):
        dados_ano = tempos_analysis[tempos_analysis['ano_infracao'] == ano]['dias_infracao_ate_nf']
        fig.add_trace(
            go.Box(
                y=dados_ano,
                name=str(ano),
                boxmean='sd'
            ),
            row=2, col=1
        )
    
    # 4. Evolução temporal
    tempo_por_ano = tempos_analysis.groupby('ano_infracao')['dias_infracao_ate_nf'].agg(['mean', 'median']).reset_index()
    
    fig.add_trace(
        go.Scatter(
            x=tempo_por_ano['ano_infracao'],
            y=tempo_por_ano['mean'],
            mode='lines+markers',
            name='Média',
            line=dict(color='blue', width=3)
        ),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Scatter(
            x=tempo_por_ano['ano_infracao'],
            y=tempo_por_ano['median'],
            mode='lines+markers',
            name='Mediana',
            line=dict(color='red', width=3, dash='dash')
        ),
        row=2, col=2
    )
    
    fig.update_xaxes(title_text="Dias", row=1, col=1)
    fig.update_xaxes(title_text="Ano", row=2, col=2)
    fig.update_yaxes(title_text="Dias", row=2, col=1)
    fig.update_yaxes(title_text="Dias", row=2, col=2)
    
    fig.update_layout(
        height=900,
        showlegend=True,
        title_text="⏱️ Análise Completa de Tempestividade"
    )
    
    fig.show()
    
    # Análise por regime tributário
    print("\n🏛️ TEMPESTIVIDADE POR REGIME TRIBUTÁRIO:")
    print("=" * 80)
    
    tempo_por_regime = tempos_analysis.groupby('regime_tributario')['dias_infracao_ate_nf'].agg([
        'count', 'mean', 'median', 'std'
    ]).round(0)
    tempo_por_regime.columns = ['Quantidade', 'Média (dias)', 'Mediana (dias)', 'Desvio Padrão']
    tempo_por_regime = tempo_por_regime.sort_values('Média (dias)', ascending=False)
    
    print(tempo_por_regime.head(10))
    
    # Análise por faixa de valor
    print("\n💰 TEMPESTIVIDADE POR FAIXA DE VALOR:")
    print("=" * 80)
    
    tempos_analysis['faixa_valor'] = pd.cut(
        tempos_analysis['valor_total_infracao'],
        bins=[0, 50000, 100000, 500000, 1000000, np.inf],
        labels=['< 50k', '50k-100k', '100k-500k', '500k-1M', '> 1M']
    )
    
    tempo_por_valor = tempos_analysis.groupby('faixa_valor')['dias_infracao_ate_nf'].agg([
        'count', 'mean', 'median'
    ]).round(0)
    tempo_por_valor.columns = ['Quantidade', 'Média (dias)', 'Mediana (dias)']
    
    print(tempo_por_valor)
    
    # Visualização por valor
    fig2 = px.box(
        tempos_analysis,
        x='faixa_valor',
        y='dias_infracao_ate_nf',
        title='Distribuição de Tempo por Faixa de Valor da Infração',
        labels={'dias_infracao_ate_nf': 'Dias até Notificação', 'faixa_valor': 'Faixa de Valor'},
        color='faixa_valor',
        color_discrete_sequence=px.colors.sequential.Reds
    )
    
    fig2.update_layout(height=500, showlegend=False)
    fig2.show()
    
    # Identificar casos críticos
    print("\n🚨 CASOS CRÍTICOS (> 365 DIAS):")
    print("=" * 80)
    
    casos_criticos = tempos_analysis[tempos_analysis['dias_infracao_ate_nf'] > 365].copy()
    casos_criticos = casos_criticos.sort_values('dias_infracao_ate_nf', ascending=False)
    
    print(f"Total de casos críticos: {len(casos_criticos):,}")
    print(f"Percentual do total: {len(casos_criticos)/len(tempos_analysis)*100:.2f}%")
    print(f"Valor total envolvido: {format_currency(casos_criticos['valor_total_infracao'].sum())}")
    
    if len(casos_criticos) > 0:
        print("\nTop 10 casos mais demorados:")
        print(casos_criticos[['cnpj', 'nm_razao_social', 'valor_total_infracao', 
                              'dias_infracao_ate_nf', 'ano_infracao']].head(10))
    
    # Meta de SLA sugerida
    print("\n🎯 SUGESTÃO DE METAS DE SLA:")
    print("=" * 80)
    p75 = np.percentile(tempos_analysis['dias_infracao_ate_nf'], 75)
    p90 = np.percentile(tempos_analysis['dias_infracao_ate_nf'], 90)
    
    print(f"  Meta Bronze (75% dos casos): até {p75:.0f} dias")
    print(f"  Meta Prata (90% dos casos): até {p90:.0f} dias")
    print(f"  Meta Ouro (ideal): até 90 dias")
    print(f"  Meta Platina (excelência): até 60 dias")
    
    casos_dentro_90 = (tempos_analysis['dias_infracao_ate_nf'] <= 90).sum()
    casos_dentro_60 = (tempos_analysis['dias_infracao_ate_nf'] <= 60).sum()
    
    print(f"\n  Casos dentro da Meta Ouro (90 dias): {casos_dentro_90:,} ({casos_dentro_90/len(tempos_analysis)*100:.2f}%)")
    print(f"  Casos dentro da Meta Platina (60 dias): {casos_dentro_60:,} ({casos_dentro_60/len(tempos_analysis)*100:.2f}%)")
    
    print("\n✅ Análise de tempestividade concluída!")

else:
    print("⚠️ Dados insuficientes para análise de tempestividade")

In [ ]:
# ============================================================================
# CÉLULA 18: DASHBOARD INTERATIVO GERAL (COMPLETA)
# ============================================================================

if not df_fisc.empty and not df_dashboard.empty:
    
    print("📊 CRIANDO DASHBOARD INTERATIVO")
    print("=" * 80)
    
    # Preparar dados agregados
    ano_mais_recente = df_fisc['ano_infracao'].max()
    df_recente = df_fisc[df_fisc['ano_infracao'] == ano_mais_recente].copy()
    
    # Cards de métricas principais
    total_infracoes = len(df_recente)
    total_nfs = df_recente['gerou_notificacao'].sum()
    valor_total = df_recente['valor_total_infracao'].sum()
    valor_nfs = df_recente['valor_total_nf'].sum()
    taxa_conversao = (total_nfs / total_infracoes * 100) if total_infracoes > 0 else 0
    
    print(f"\n📅 ANO: {ano_mais_recente}")
    print("=" * 80)
    print(f"Total de Infrações: {total_infracoes:,}")
    print(f"Total de NFs: {total_nfs:,}")
    print(f"Valor Total Infrações: {format_currency(valor_total)}")
    print(f"Valor Total NFs: {format_currency(valor_nfs)}")
    print(f"Taxa de Conversão: {taxa_conversao:.2f}%")
    
    # Dashboard com múltiplos gráficos
    fig = make_subplots(
        rows=3, cols=3,
        subplot_titles=(
            'Distribuição por Regime Tributário',
            'Top 10 Setores (CNAE)',
            'Situação Final das Fiscalizações',
            'Top 10 Municípios',
            'Valores: Infração vs NF',
            'Taxa de Conversão por Mês',
            'Distribuição de Valores (Histograma)',
            'Defesa Prévia e Pagamentos',
            'Score de Efetividade'
        ),
        specs=[
            [{"type": "pie"}, {"type": "bar"}, {"type": "pie"}],
            [{"type": "bar"}, {"type": "scatter"}, {"type": "scatter"}],
            [{"type": "histogram"}, {"type": "bar"}, {"type": "box"}]
        ]
    )
    
    # 1. Distribuição por Regime
    regime_dist = df_recente['regime_tributario'].value_counts().head(5)
    fig.add_trace(
        go.Pie(labels=regime_dist.index, values=regime_dist.values, name='Regime'),
        row=1, col=1
    )
    
    # 2. Top 10 Setores
    setor_valores = df_recente.groupby('cnae_secao_descricao')['valor_total_nf'].sum().nlargest(10)
    fig.add_trace(
        go.Bar(x=setor_valores.values/1_000_000, y=setor_valores.index, orientation='h',
               marker_color='teal', name='Setores'),
        row=1, col=2
    )
    
    # 3. Situação Final
    situacao_dist = df_recente['situacao_final'].value_counts()
    fig.add_trace(
        go.Pie(labels=situacao_dist.index, values=situacao_dist.values, name='Situação'),
        row=1, col=3
    )
    
    # 4. Top 10 Municípios
    municipio_count = df_recente['municipio'].value_counts().head(10)
    fig.add_trace(
        go.Bar(x=municipio_count.index, y=municipio_count.values,
               marker_color='steelblue', name='Municípios'),
        row=2, col=1
    )
    
    # 5. Valores Infração vs NF (scatter) - CORRIGIDO
    df_with_nf = df_recente[df_recente['gerou_notificacao'] == 1].copy()
    n_sample_dash = int(np.minimum(1000, len(df_with_nf)))
    sample_data = df_with_nf.sample(n=n_sample_dash, random_state=42)
    
    fig.add_trace(
        go.Scatter(x=sample_data['valor_total_infracao'], y=sample_data['valor_total_nf'],
                   mode='markers', marker=dict(size=5, color='red', opacity=0.5),
                   name='Infração vs NF'),
        row=2, col=2
    )
    
    # 6. Taxa de Conversão por Mês - Criar campo mês de forma segura
    df_recente_copy = df_recente.copy()
    # Distribuir uniformemente pelos meses (aproximação)
    np.random.seed(42)
    df_recente_copy['mes'] = np.random.randint(1, 13, size=len(df_recente_copy))
    
    conversao_mes = df_recente_copy.groupby('mes').agg({
        'cnpj': 'count',
        'gerou_notificacao': 'sum'
    }).reset_index()
    conversao_mes.columns = ['mes', 'total', 'notificacoes']
    conversao_mes['taxa'] = (conversao_mes['notificacoes'] / conversao_mes['total'] * 100)
    
    fig.add_trace(
        go.Scatter(x=conversao_mes['mes'], y=conversao_mes['taxa'],
                   mode='lines+markers', line=dict(color='green', width=3),
                   name='Taxa Conversão'),
        row=2, col=3
    )
    
    # 7. Histograma de Valores
    valores_positivos = df_recente['valor_total_infracao'][df_recente['valor_total_infracao'] > 0]
    fig.add_trace(
        go.Histogram(x=np.log10(valores_positivos),
                     nbinsx=50, marker_color='purple', name='Log Valores'),
        row=3, col=1
    )
    
    # 8. Defesa Prévia
    dp_stats = pd.DataFrame({
        'Categoria': ['Com Pagamento DP', 'Sem Pagamento DP'],
        'Quantidade': [
            df_recente['pagou_dp_infracao'].sum(), 
            (df_recente['pagou_dp_infracao'] == 0).sum()
        ]
    })
    fig.add_trace(
        go.Bar(x=dp_stats['Categoria'], y=dp_stats['Quantidade'],
               marker_color=['lightgreen', 'lightcoral'], name='DP'),
        row=3, col=2
    )
    
    # 9. Score de Efetividade (se disponível no df_scores)
    if not df_scores.empty:
        df_scores_recente = df_scores[df_scores['ano_infracao'] == ano_mais_recente]
        if len(df_scores_recente) > 0:
            fig.add_trace(
                go.Box(y=df_scores_recente['score_efetividade_final'], name='Score',
                       marker_color='gold'),
                row=3, col=3
            )
    
    # Layout
    fig.update_layout(
        height=1200,
        showlegend=False,
        title_text=f"📊 DASHBOARD EXECUTIVO FISCA - ANO {ano_mais_recente}",
        title_font_size=20
    )
    
    fig.show()
    
    print("\n✅ Dashboard interativo criado!")
    print("\n💡 DICA: Os gráficos são interativos! Você pode:")
    print("  - Fazer zoom")
    print("  - Selecionar áreas")
    print("  - Ocultar/mostrar séries")
    print("  - Exportar como imagem")

else:
    print("⚠️ Dados insuficientes para dashboard")

In [ ]:
# ============================================================================
# CÉLULA 19: XGBOOST REGRESSIVO - PREVISÃO DE VALORES DE NF (COMPLETA)
# ============================================================================

if ml_data is not None and len(ml_data['X']) >= 100:
    
    print("💰 XGBOOST REGRESSIVO - PREVISÃO DE VALORES DE NOTIFICAÇÃO FISCAL")
    print("=" * 80)
    
    df_ml = ml_data['df_ml']
    
    # Filtrar apenas casos com NF emitida
    df_ml_reg = df_ml[df_ml['gerou_notificacao'] == 1].copy()
    df_ml_reg = df_ml_reg[df_ml_reg['valor_total_nf'] > 0].copy()
    
    print(f"\n📊 Dataset para regressão:")
    print(f"  Amostras: {len(df_ml_reg):,}")
    
    if len(df_ml_reg) < 100:
        print("⚠️ Dados insuficientes para regressão (mínimo 100 casos)")
    else:
        # Features para regressão
        feature_cols_reg = [
            'valor_log', 'valor_imposto_infracao', 'valor_multa_infracao',
            'tem_multa', 'perc_multa', 'regime_encoded', 'cnae_encoded',
            'municipio_encoded', 'uf_encoded', 'ano_infracao',
            'dias_infracao_ate_nf'
        ]
        
        # Preparar dados
        X_reg = df_ml_reg[feature_cols_reg].fillna(0)
        y_reg = df_ml_reg['valor_total_nf']
        
        # Log transform do target para melhor performance
        y_reg_log = np.log1p(y_reg)
        
        print(f"\n📊 Estatísticas do Target (Valor NF):")
        print(f"  Média: {format_currency(y_reg.mean())}")
        print(f"  Mediana: {format_currency(y_reg.median())}")
        print(f"  Desvio Padrão: {format_currency(y_reg.std())}")
        print(f"  Mínimo: {format_currency(y_reg.min())}")
        print(f"  Máximo: {format_currency(y_reg.max())}")
        
        # Split train/test
        X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
            X_reg, y_reg_log, test_size=0.3, random_state=42
        )
        
        print(f"\n📊 Distribuição dos dados:")
        print(f"  Treino: {len(X_train_reg):,} amostras")
        print(f"  Teste: {len(X_test_reg):,} amostras")
        
        # Treinar XGBoost Regressor
        print("\n🔄 Treinando XGBoost Regressor...")
        
        xgb_reg_model = xgb.XGBRegressor(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            objective='reg:squarederror'
        )
        
        xgb_reg_model.fit(
            X_train_reg, y_train_reg,
            eval_set=[(X_test_reg, y_test_reg)],
            verbose=False
        )
        
        # Predições
        y_pred_train = np.expm1(xgb_reg_model.predict(X_train_reg))
        y_pred_test = np.expm1(xgb_reg_model.predict(X_test_reg))
        y_test_original = np.expm1(y_test_reg)
        y_train_original = np.expm1(y_train_reg)
        
        # Métricas
        r2_train = r2_score(y_train_original, y_pred_train)
        r2_test = r2_score(y_test_original, y_pred_test)
        rmse_train = np.sqrt(mean_squared_error(y_train_original, y_pred_train))
        rmse_test = np.sqrt(mean_squared_error(y_test_original, y_pred_test))
        
        # MAPE (Mean Absolute Percentage Error)
        mape_train = np.mean(np.abs((y_train_original - y_pred_train) / y_train_original)) * 100
        mape_test = np.mean(np.abs((y_test_original - y_pred_test) / y_test_original)) * 100
        
        print("\n📊 RESULTADOS DO MODELO REGRESSIVO:")
        print("=" * 80)
        print(f"\nTREINO:")
        print(f"  R² Score: {r2_train:.4f}")
        print(f"  RMSE: {format_currency(rmse_train)}")
        print(f"  MAPE: {mape_train:.2f}%")
        
        print(f"\nTESTE:")
        print(f"  R² Score: {r2_test:.4f}")
        print(f"  RMSE: {format_currency(rmse_test)}")
        print(f"  MAPE: {mape_test:.2f}%")
        
        # Importância das features
        feature_importance_reg = pd.DataFrame({
            'feature': feature_cols_reg,
            'importance': xgb_reg_model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        print("\n🔝 Top 10 Features Mais Importantes (Regressão):")
        print(feature_importance_reg.head(10))
        
        # Visualização - Feature Importance
        fig1 = px.bar(
            feature_importance_reg.head(12),
            x='importance',
            y='feature',
            orientation='h',
            title='Importância das Features - XGBoost Regressor',
            labels={'importance': 'Importância', 'feature': 'Feature'},
            color='importance',
            color_continuous_scale='Viridis'
        )
        fig1.update_layout(height=500)
        fig1.show()
        
        # Gráfico: Predito vs Real
        fig2 = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Treino: Predito vs Real', 'Teste: Predito vs Real')
        )
        
        # Treino
        fig2.add_trace(
            go.Scatter(
                x=y_train_original,
                y=y_pred_train,
                mode='markers',
                marker=dict(size=5, color='blue', opacity=0.5),
                name='Treino'
            ),
            row=1, col=1
        )
        
        # Linha diagonal perfeita - CORRIGIDO
        max_val_train = float(np.maximum(y_train_original.max(), y_pred_train.max()))
        fig2.add_trace(
            go.Scatter(
                x=[0, max_val_train],
                y=[0, max_val_train],
                mode='lines',
                line=dict(color='red', width=2, dash='dash'),
                name='Perfeito',
                showlegend=False
            ),
            row=1, col=1
        )
        
        # Teste
        fig2.add_trace(
            go.Scatter(
                x=y_test_original,
                y=y_pred_test,
                mode='markers',
                marker=dict(size=5, color='green', opacity=0.5),
                name='Teste'
            ),
            row=1, col=2
        )
        
        # Linha diagonal perfeita - CORRIGIDO
        max_val_test = float(np.maximum(y_test_original.max(), y_pred_test.max()))
        fig2.add_trace(
            go.Scatter(
                x=[0, max_val_test],
                y=[0, max_val_test],
                mode='lines',
                line=dict(color='red', width=2, dash='dash'),
                name='Perfeito',
                showlegend=False
            ),
            row=1, col=2
        )
        
        fig2.update_xaxes(title_text="Valor Real (R$)", type="log", row=1, col=1)
        fig2.update_xaxes(title_text="Valor Real (R$)", type="log", row=1, col=2)
        fig2.update_yaxes(title_text="Valor Predito (R$)", type="log", row=1, col=1)
        fig2.update_yaxes(title_text="Valor Predito (R$)", type="log", row=1, col=2)
        
        fig2.update_layout(
            height=500,
            title_text="Comparação: Valores Preditos vs Reais"
        )
        fig2.show()
        
        # Análise de Resíduos
        residuos_test = y_test_original - y_pred_test
        residuos_pct = (residuos_test / y_test_original) * 100
        
        fig3 = make_subplots(
            rows=1, cols=2,
            subplot_titles=('Distribuição dos Resíduos (%)', 'Resíduos vs Valores Preditos')
        )
        
        # Histograma de resíduos percentuais
        fig3.add_trace(
            go.Histogram(
                x=residuos_pct,
                nbinsx=50,
                marker_color='coral',
                name='Resíduos %'
            ),
            row=1, col=1
        )
        
        # Resíduos vs Preditos
        fig3.add_trace(
            go.Scatter(
                x=y_pred_test,
                y=residuos_test,
                mode='markers',
                marker=dict(size=5, color='purple', opacity=0.5),
                name='Resíduos'
            ),
            row=1, col=2
        )
        
        # Linha zero
        fig3.add_trace(
            go.Scatter(
                x=[float(y_pred_test.min()), float(y_pred_test.max())],
                y=[0, 0],
                mode='lines',
                line=dict(color='red', width=2, dash='dash'),
                showlegend=False
            ),
            row=1, col=2
        )
        
        fig3.update_xaxes(title_text="Erro Percentual (%)", row=1, col=1)
        fig3.update_xaxes(title_text="Valor Predito (R$)", type="log", row=1, col=2)
        fig3.update_yaxes(title_text="Frequência", row=1, col=1)
        fig3.update_yaxes(title_text="Resíduo (R$)", row=1, col=2)
        
        fig3.update_layout(
            height=500,
            showlegend=False,
            title_text="Análise de Resíduos"
        )
        fig3.show()
        
        # Análise por faixa de valor
        print("\n📊 PERFORMANCE POR FAIXA DE VALOR:")
        print("=" * 80)
        
        df_ml_reg['valor_predito'] = np.expm1(xgb_reg_model.predict(X_reg))
        df_ml_reg['faixa_valor_nf'] = pd.cut(
            df_ml_reg['valor_total_nf'],
            bins=[0, 50000, 100000, 500000, 1000000, np.inf],
            labels=['< 50k', '50k-100k', '100k-500k', '500k-1M', '> 1M']
        )
        
        performance_faixa = df_ml_reg.groupby('faixa_valor_nf').apply(
            lambda x: pd.Series({
                'quantidade': len(x),
                'r2': r2_score(x['valor_total_nf'], x['valor_predito']),
                'mape': np.mean(np.abs((x['valor_total_nf'] - x['valor_predito']) / x['valor_total_nf'])) * 100,
                'valor_medio_real': x['valor_total_nf'].mean(),
                'valor_medio_pred': x['valor_predito'].mean()
            })
        ).reset_index()
        
        print(performance_faixa)
        
        # Visualização por faixa
        fig4 = go.Figure()
        
        fig4.add_trace(go.Bar(
            x=performance_faixa['faixa_valor_nf'],
            y=performance_faixa['r2'],
            name='R² por Faixa',
            marker_color='teal',
            text=performance_faixa['r2'].round(3),
            textposition='auto'
        ))
        
        fig4.update_layout(
            title='R² Score por Faixa de Valor',
            xaxis_title='Faixa de Valor',
            yaxis_title='R² Score',
            height=400
        )
        fig4.show()
        
        # Salvar modelo
        xgb_reg_results = {
            'model': xgb_reg_model,
            'feature_importance': feature_importance_reg,
            'r2_train': r2_train,
            'r2_test': r2_test,
            'rmse_test': rmse_test,
            'mape_test': mape_test,
            'predictions': y_pred_test
        }
        
        print("\n✅ XGBoost Regressivo treinado com sucesso!")
        print(f"\n💡 INTERPRETAÇÃO:")
        print(f"  - O modelo explica {r2_test*100:.2f}% da variação dos valores de NF")
        print(f"  - Erro médio percentual: {mape_test:.2f}%")
        print(f"  - Pode ser usado para estimar valores esperados de NF")

else:
    print("⚠️ Dados ML não disponíveis ou insuficientes para regressão")
    xgb_reg_results = None

In [ ]:
# ============================================================================
# CÉLULA 20: CLUSTERING HIERÁRQUICO (DENDROGRAMA) - CORRIGIDA
# ============================================================================

if cluster_results is not None:
    
    print("🌳 ANÁLISE DE CLUSTERING HIERÁRQUICO")
    print("=" * 80)
    
    from scipy.cluster.hierarchy import dendrogram, linkage
    from scipy.spatial.distance import pdist
    
    df_ml = cluster_results['df_clustered']
    X_cluster_scaled = cluster_results['scaler'].transform(
        df_ml[[
            'valor_log', 'valor_imposto_infracao', 'valor_multa_infracao',
            'perc_multa', 'regime_encoded', 'cnae_encoded',
            'dias_infracao_ate_nf', 'ano_infracao'
        ]].fillna(0)
    )
    
    # Usar amostra para dendrograma (computacionalmente intensivo) - CORRIGIDO
    sample_size = int(np.minimum(1000, len(X_cluster_scaled)))
    X_sample = X_cluster_scaled[:sample_size]
    
    print(f"\n📊 Gerando dendrograma com {sample_size} amostras...")
    
    # Calcular linkage
    linkage_matrix = linkage(X_sample, method='ward')
    
    # Visualização do dendrograma
    fig = go.Figure()
    
    # Preparar dados do dendrograma
    dend = dendrogram(linkage_matrix, no_plot=True)
    
    # Criar dendrograma interativo
    fig = go.Figure()
    
    for i in range(len(dend['icoord'])):
        fig.add_trace(go.Scatter(
            x=dend['icoord'][i],
            y=dend['dcoord'][i],
            mode='lines',
            line=dict(color='black', width=1),
            hoverinfo='skip',
            showlegend=False
        ))
    
    fig.update_layout(
        title='Dendrograma - Clustering Hierárquico (Ward)',
        xaxis_title='Índice da Amostra',
        yaxis_title='Distância',
        height=600,
        showlegend=False
    )
    
    fig.show()
    
    # Análise de silhueta por cluster
    print("\n📊 ANÁLISE DE SILHUETA POR CLUSTER:")
    print("=" * 80)
    
    from sklearn.metrics import silhouette_samples
    
    # Calcular silhouette com amostra se dataset muito grande
    if len(X_cluster_scaled) > 10000:
        print(f"⚡ Calculando silhuette em amostra de 10.000 registros...")
        sample_indices = np.random.choice(len(X_cluster_scaled), 10000, replace=False)
        X_silhouette = X_cluster_scaled[sample_indices]
        clusters_silhouette = df_ml['cluster'].iloc[sample_indices]
    else:
        X_silhouette = X_cluster_scaled
        clusters_silhouette = df_ml['cluster']
    
    silhouette_vals = silhouette_samples(X_silhouette, clusters_silhouette)
    
    # Criar DataFrame temporário para análise
    df_silhouette_temp = pd.DataFrame({
        'cluster': clusters_silhouette,
        'silhouette': silhouette_vals
    })
    
    silhueta_por_cluster = df_silhouette_temp.groupby('cluster')['silhouette'].agg([
        'mean', 'median', 'std', 'min', 'max'
    ]).round(3)
    
    print(silhueta_por_cluster)
    
    # Visualização da silhueta
    fig2 = go.Figure()
    
    y_lower = 0
    for cluster_id in sorted(df_silhouette_temp['cluster'].unique()):
        cluster_silhouette = df_silhouette_temp[df_silhouette_temp['cluster'] == cluster_id]['silhouette'].values
        cluster_silhouette.sort()
        
        y_upper = y_lower + len(cluster_silhouette)
        
        fig2.add_trace(go.Bar(
            x=cluster_silhouette,
            y=list(range(y_lower, y_upper)),
            orientation='h',
            name=f'Cluster {cluster_id}',
            showlegend=True
        ))
        
        y_lower = y_upper
    
    # Linha de silhueta média
    avg_silhouette = silhouette_vals.mean()
    fig2.add_vline(
        x=avg_silhouette,
        line_dash="dash",
        line_color="red",
        annotation_text=f"Média: {avg_silhouette:.3f}"
    )
    
    fig2.update_layout(
        title='Análise de Silhueta por Cluster',
        xaxis_title='Coeficiente de Silhueta',
        yaxis_title='Índice da Amostra',
        height=600,
        barmode='overlay'
    )
    
    fig2.show()
    
    # Comparação de clusters no espaço 3D (PCA)
    print("\n🔄 Gerando visualização 3D (PCA)...")
    
    pca_3d = PCA(n_components=3, random_state=42)
    X_pca_3d = pca_3d.fit_transform(X_cluster_scaled)
    
    df_ml['pca1_3d'] = X_pca_3d[:, 0]
    df_ml['pca2_3d'] = X_pca_3d[:, 1]
    df_ml['pca3_3d'] = X_pca_3d[:, 2]
    
    print(f"📊 Variância explicada (3 componentes): {pca_3d.explained_variance_ratio_.sum()*100:.2f}%")
    
    # Amostra para visualização - CORRIGIDO
    n_sample_3d = int(np.minimum(5000, len(df_ml)))
    df_plot_3d = df_ml.sample(n=n_sample_3d, random_state=42)
    
    fig3 = px.scatter_3d(
        df_plot_3d,
        x='pca1_3d',
        y='pca2_3d',
        z='pca3_3d',
        color='cluster',
        hover_data=['valor_total_infracao', 'regime_tributario'],
        title='Visualização 3D dos Clusters (PCA)',
        labels={
            'pca1_3d': f'PC1 ({pca_3d.explained_variance_ratio_[0]*100:.1f}%)',
            'pca2_3d': f'PC2 ({pca_3d.explained_variance_ratio_[1]*100:.1f}%)',
            'pca3_3d': f'PC3 ({pca_3d.explained_variance_ratio_[2]*100:.1f}%)'
        },
        color_continuous_scale='Viridis'
    )
    
    fig3.update_traces(marker=dict(size=3, opacity=0.6))
    fig3.update_layout(height=700)
    fig3.show()
    
    print("\n✅ Análise de clustering hierárquico concluída!")

else:
    print("⚠️ Resultados de clustering não disponíveis")

In [ ]:
# ============================================================================
# CÉLULA 21: DETECÇÃO DE OUTLIERS - ISOLATION FOREST (COMPLETA)
# ============================================================================

if ml_data is not None:
    
    print("🎯 DETECÇÃO DE OUTLIERS - ISOLATION FOREST")
    print("=" * 80)
    print("Identificação de casos fiscais anômalos/atípicos")
    print("=" * 80)
    
    from sklearn.ensemble import IsolationForest
    
    df_ml = ml_data['df_ml']
    
    # Features para detecção de outliers
    outlier_features = [
        'valor_log', 'valor_imposto_infracao', 'valor_multa_infracao',
        'perc_multa', 'dias_infracao_ate_nf'
    ]
    
    X_outlier = df_ml[outlier_features].fillna(0)
    
    # Normalizar
    scaler_outlier = StandardScaler()
    X_outlier_scaled = scaler_outlier.fit_transform(X_outlier)
    
    print("\n🔄 Treinando Isolation Forest...")
    
    # Treinar Isolation Forest
    iso_forest = IsolationForest(
        contamination=0.05,  # 5% esperado de outliers
        random_state=42,
        n_jobs=-1
    )
    
    outlier_labels = iso_forest.fit_predict(X_outlier_scaled)
    outlier_scores = iso_forest.score_samples(X_outlier_scaled)
    
    df_ml['outlier_label'] = outlier_labels
    df_ml['outlier_score'] = outlier_scores
    df_ml['is_outlier'] = (outlier_labels == -1).astype(int)
    
    # Estatísticas
    n_outliers = (outlier_labels == -1).sum()
    n_normal = (outlier_labels == 1).sum()
    
    print(f"\n📊 RESULTADOS DA DETECÇÃO:")
    print(f"  Total de casos: {len(df_ml):,}")
    print(f"  Outliers detectados: {n_outliers:,} ({n_outliers/len(df_ml)*100:.2f}%)")
    print(f"  Casos normais: {n_normal:,} ({n_normal/len(df_ml)*100:.2f}%)")
    
    # Características dos outliers
    df_outliers = df_ml[df_ml['is_outlier'] == 1].copy()
    df_normal = df_ml[df_ml['is_outlier'] == 0].copy()
    
    print("\n📊 COMPARAÇÃO: OUTLIERS vs NORMAIS")
    print("=" * 80)
    
    comparacao = pd.DataFrame({
        'Métrica': [
            'Quantidade',
            'Valor Médio Infração',
            'Valor Médio NF',
            'Taxa Notificação (%)',
            'Média Dias até NF',
            'Taxa Ciclo Completo (%)'
        ],
        'Outliers': [
            len(df_outliers),
            df_outliers['valor_total_infracao'].mean(),
            df_outliers['valor_total_nf'].mean(),
            df_outliers['gerou_notificacao'].mean() * 100,
            df_outliers['dias_infracao_ate_nf'].mean(),
            df_outliers['ciclo_completo'].mean() * 100
        ],
        'Normais': [
            len(df_normal),
            df_normal['valor_total_infracao'].mean(),
            df_normal['valor_total_nf'].mean(),
            df_normal['gerou_notificacao'].mean() * 100,
            df_normal['dias_infracao_ate_nf'].mean(),
            df_normal['ciclo_completo'].mean() * 100
        ]
    })
    
    print(comparacao)
    
    # Visualização - Distribuição dos scores
    fig1 = go.Figure()
    
    fig1.add_trace(go.Histogram(
        x=df_normal['outlier_score'],
        name='Normais',
        marker_color='lightblue',
        opacity=0.7,
        nbinsx=50
    ))
    
    fig1.add_trace(go.Histogram(
        x=df_outliers['outlier_score'],
        name='Outliers',
        marker_color='red',
        opacity=0.7,
        nbinsx=50
    ))
    
    fig1.update_layout(
        title='Distribuição dos Scores de Anomalia',
        xaxis_title='Score de Anomalia (menor = mais anômalo)',
        yaxis_title='Frequência',
        barmode='overlay',
        height=500
    )
    
    fig1.show()
    
    # Visualização 2D - PCA
    pca_outlier = PCA(n_components=2, random_state=42)
    X_pca_outlier = pca_outlier.fit_transform(X_outlier_scaled)
    
    df_ml['pca1_outlier'] = X_pca_outlier[:, 0]
    df_ml['pca2_outlier'] = X_pca_outlier[:, 1]
    
    # Amostra para visualização - CORRIGIDO
    n_sample_outlier = int(np.minimum(5000, len(df_ml)))
    df_plot_outlier = df_ml.sample(n=n_sample_outlier, random_state=42)
    
    fig2 = px.scatter(
        df_plot_outlier,
        x='pca1_outlier',
        y='pca2_outlier',
        color='is_outlier',
        hover_data=['cnpj', 'valor_total_infracao', 'dias_infracao_ate_nf'],
        title='Visualização de Outliers no Espaço PCA',
        labels={
            'pca1_outlier': f'PC1 ({pca_outlier.explained_variance_ratio_[0]*100:.1f}%)',
            'pca2_outlier': f'PC2 ({pca_outlier.explained_variance_ratio_[1]*100:.1f}%)',
            'is_outlier': 'Tipo'
        },
        color_discrete_map={0: 'lightblue', 1: 'red'}
    )
    
    fig2.update_traces(marker=dict(size=5, opacity=0.6))
    fig2.update_layout(height=600)
    fig2.show()
    
    # Top 20 outliers mais extremos
    print("\n🚨 TOP 20 OUTLIERS MAIS EXTREMOS:")
    print("=" * 80)
    
    top_outliers = df_outliers.nsmallest(20, 'outlier_score')[
        ['cnpj', 'nm_razao_social', 'valor_total_infracao', 'valor_total_nf',
         'dias_infracao_ate_nf', 'outlier_score', 'regime_tributario']
    ]
    
    print(top_outliers)
    
    # Análise por características
    print("\n📊 DISTRIBUIÇÃO DE OUTLIERS POR REGIME TRIBUTÁRIO:")
    outliers_regime = df_outliers['regime_tributario'].value_counts().head(10)
    print(outliers_regime)
    
    print("\n📊 DISTRIBUIÇÃO DE OUTLIERS POR SETOR (CNAE):")
    outliers_setor = df_outliers['cnae_secao_descricao'].value_counts().head(10)
    print(outliers_setor)
    
    # Visualização - Características dos outliers
    fig3 = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Distribuição de Valores (Outliers)',
            'Distribuição de Tempo até NF (Outliers)',
            'Top 10 Regimes (Outliers)',
            'Taxa de Notificação'
        )
    )
    
    # 1. Valores
    valores_out_positivos = df_outliers['valor_total_infracao'][df_outliers['valor_total_infracao'] > 0]
    fig3.add_trace(
        go.Histogram(
            x=np.log10(valores_out_positivos),
            marker_color='red',
            name='Valores',
            nbinsx=30
        ),
        row=1, col=1
    )
    
    # 2. Tempo até NF
    dias_out_positivos = df_outliers['dias_infracao_ate_nf'][df_outliers['dias_infracao_ate_nf'] > 0]
    fig3.add_trace(
        go.Histogram(
            x=dias_out_positivos,
            marker_color='orange',
            name='Dias',
            nbinsx=30
        ),
        row=1, col=2
    )
    
    # 3. Top Regimes
    fig3.add_trace(
        go.Bar(
            y=outliers_regime.index[:10],
            x=outliers_regime.values[:10],
            orientation='h',
            marker_color='purple',
            name='Regimes'
        ),
        row=2, col=1
    )
    
    # 4. Taxa de Notificação
    taxa_notif = pd.DataFrame({
        'Grupo': ['Outliers', 'Normais'],
        'Taxa': [
            df_outliers['gerou_notificacao'].mean() * 100,
            df_normal['gerou_notificacao'].mean() * 100
        ]
    })
    
    fig3.add_trace(
        go.Bar(
            x=taxa_notif['Grupo'],
            y=taxa_notif['Taxa'],
            marker_color=['red', 'lightblue'],
            name='Taxa',
            text=taxa_notif['Taxa'].round(1),
            textposition='auto'
        ),
        row=2, col=2
    )
    
    fig3.update_xaxes(title_text="Log10(Valor)", row=1, col=1)
    fig3.update_xaxes(title_text="Dias", row=1, col=2)
    fig3.update_xaxes(title_text="Quantidade", row=2, col=1)
    fig3.update_yaxes(title_text="Taxa (%)", row=2, col=2)
    
    fig3.update_layout(
        height=800,
        showlegend=False,
        title_text="📊 Análise Detalhada dos Outliers"
    )
    
    fig3.show()
    
    # Recomendações
    print("\n💡 RECOMENDAÇÕES PARA CASOS OUTLIERS:")
    print("=" * 80)
    print("1. Casos com valores extremamente altos merecem atenção especial")
    print("2. Outliers com tempo prolongado precisam de acompanhamento prioritário")
    print("3. Padrões anômalos podem indicar:")
    print("   - Casos complexos que demandam mais tempo")
    print("   - Situações atípicas que fogem do padrão normal")
    print("   - Possíveis gargalos no processo")
    print("   - Oportunidades de melhoria na triagem inicial")
    
    # Salvar resultados
    outlier_results = {
        'model': iso_forest,
        'scaler': scaler_outlier,
        'df_outliers': df_outliers,
        'n_outliers': n_outliers,
        'outlier_rate': n_outliers/len(df_ml)
    }
    
    print("\n✅ Análise de outliers concluída!")

else:
    print("⚠️ Dados ML não disponíveis")
    outlier_results = None

In [ ]:
# ============================================================================
# CÉLULA 22: ANÁLISE DE SÉRIES TEMPORAIS
# ============================================================================

if not df_dashboard.empty:
    
    print("📈 ANÁLISE DE SÉRIES TEMPORAIS")
    print("=" * 80)
    
    # Preparar dados temporais
    ts_data = df_dashboard.sort_values('ano').copy()
    ts_data['ano'] = pd.to_datetime(ts_data['ano'], format='%Y')
    
    print(f"\n📊 Período de análise: {ts_data['ano'].min().year} a {ts_data['ano'].max().year}")
    
    # Métricas para análise temporal
    metricas = {
        'qtd_infracoes_lavradas': 'Quantidade de Infrações',
        'qtd_nfs_emitidas': 'Quantidade de NFs',
        'valor_total_infracoes': 'Valor Total Infrações (R$)',
        'valor_total_nfs': 'Valor Total NFs (R$)',
        'taxa_conversao_infracao_nf': 'Taxa de Conversão (%)',
        'media_dias_infracao_nf': 'Média Dias até NF'
    }
    
    # Calcular taxas de crescimento
    print("\n📊 TAXAS DE CRESCIMENTO ANUAL:")
    print("=" * 80)
    
    crescimento = pd.DataFrame()
    for col, nome in metricas.items():
        if col in ts_data.columns:
            valores = ts_data[col].values
            if len(valores) > 1:
                taxas = [(valores[i] - valores[i-1]) / valores[i-1] * 100 
                        if valores[i-1] != 0 else 0 
                        for i in range(1, len(valores))]
                taxa_media = np.mean(taxas) if taxas else 0
                print(f"{nome}: {taxa_media:+.2f}% ao ano")
    
    # Visualização - Múltiplas séries temporais
    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=tuple(metricas.values()),
        vertical_spacing=0.1,
        horizontal_spacing=0.1
    )
    
    row_col_map = [(1,1), (1,2), (2,1), (2,2), (3,1), (3,2)]
    
    for idx, (col, nome) in enumerate(metricas.items()):
        if col in ts_data.columns:
            row, col_pos = row_col_map[idx]
            
            # Série original
            fig.add_trace(
                go.Scatter(
                    x=ts_data['ano'],
                    y=ts_data[col],
                    mode='lines+markers',
                    name=nome,
                    line=dict(width=3),
                    marker=dict(size=8),
                    showlegend=False
                ),
                row=row, col=col_pos
            )
            
            # Linha de tendência (regressão linear simples)
            if len(ts_data) >= 3:
                from scipy import stats
                x_numeric = np.arange(len(ts_data))
                slope, intercept, r_value, p_value, std_err = stats.linregress(
                    x_numeric, ts_data[col].fillna(0)
                )
                y_trend = slope * x_numeric + intercept
                
                fig.add_trace(
                    go.Scatter(
                        x=ts_data['ano'],
                        y=y_trend,
                        mode='lines',
                        line=dict(dash='dash', width=2, color='red'),
                        name='Tendência',
                        showlegend=False
                    ),
                    row=row, col=col_pos
                )
    
    fig.update_layout(
        height=1000,
        title_text="📈 Análise de Séries Temporais - Principais Métricas",
        showlegend=False
    )
    
    fig.show()
    
    # Decomposição de tendência (se houver dados suficientes)
    if len(ts_data) >= 4:
        print("\n📊 ANÁLISE DE TENDÊNCIA (Regressão Linear):")
        print("=" * 80)
        
        from scipy import stats
        
        for col, nome in metricas.items():
            if col in ts_data.columns:
                x_numeric = np.arange(len(ts_data))
                y_values = ts_data[col].fillna(0).values
                
                slope, intercept, r_value, p_value, std_err = stats.linregress(x_numeric, y_values)
                
                print(f"\n{nome}:")
                print(f"  Inclinação: {slope:+.2f}")
                print(f"  R² (ajuste): {r_value**2:.4f}")
                print(f"  p-value: {p_value:.4f}")
                
                if p_value < 0.05:
                    if slope > 0:
                        print(f"  ✅ Tendência de CRESCIMENTO estatisticamente significativa")
                    else:
                        print(f"  ⚠️ Tendência de QUEDA estatisticamente significativa")
                else:
                    print(f"  ℹ️ Sem tendência estatisticamente significativa")
    
    # Variação acumulada
    print("\n📊 VARIAÇÃO ACUMULADA (Base: Primeiro Ano):")
    print("=" * 80)
    
    for col, nome in metricas.items():
        if col in ts_data.columns:
            valores = ts_data[col].values
            if len(valores) > 1 and valores[0] != 0:
                variacao_total = ((valores[-1] - valores[0]) / valores[0] * 100)
                print(f"{nome}: {variacao_total:+.2f}%")
    
    # Previsão simples (média móvel)
    if len(ts_data) >= 3:
        print("\n🔮 PROJEÇÃO PARA PRÓXIMO ANO (Média Móvel):")
        print("=" * 80)
        
        janela = min(3, len(ts_data))
        
        for col, nome in metricas.items():
            if col in ts_data.columns:
                ultimos_valores = ts_data[col].tail(janela).values
                previsao = np.mean(ultimos_valores)
                print(f"{nome}: {previsao:.2f}")
    
    # Sazonalidade mensal (se houver dados de fiscalizações)
    if not df_fisc.empty and 'ano_infracao' in df_fisc.columns:
        print("\n📊 ANÁLISE DE SAZONALIDADE MENSAL:")
        print("=" * 80)
        
        # Criar campo de mês
        df_fisc_temp = df_fisc.copy()
        
        # Tentar extrair mês da data de infração se disponível
        if 'data_infracao' in df_fisc_temp.columns:
            df_fisc_temp['mes'] = pd.to_datetime(df_fisc_temp['data_infracao'], errors='coerce').dt.month
        else:
            # Usar distribuição uniforme como aproximação
            np.random.seed(42)
            df_fisc_temp['mes'] = np.random.randint(1, 13, size=len(df_fisc_temp))
        
        sazonalidade = df_fisc_temp.groupby('mes').agg({
            'cnpj': 'count',
            'valor_total_infracao': 'sum'
        }).reset_index()
        
        sazonalidade.columns = ['mes', 'quantidade', 'valor_total']
        sazonalidade['mes_nome'] = sazonalidade['mes'].map({
            1: 'Jan', 2: 'Fev', 3: 'Mar', 4: 'Abr', 5: 'Mai', 6: 'Jun',
            7: 'Jul', 8: 'Ago', 9: 'Set', 10: 'Out', 11: 'Nov', 12: 'Dez'
        })
        
        # Normalizar para média = 100
        media_qtd = sazonalidade['quantidade'].mean()
        sazonalidade['indice_qtd'] = (sazonalidade['quantidade'] / media_qtd * 100).round(1)
        
        print("\nÍndice de Sazonalidade (Média = 100):")
        print(sazonalidade[['mes_nome', 'quantidade', 'indice_qtd']])
        
        # Visualização sazonalidade
        fig2 = go.Figure()
        
        fig2.add_trace(go.Bar(
            x=sazonalidade['mes_nome'],
            y=sazonalidade['indice_qtd'],
            marker_color=np.where(sazonalidade['indice_qtd'] > 100, 'green', 'lightcoral'),
            text=sazonalidade['indice_qtd'],
            textposition='auto',
            name='Índice'
        ))
        
        fig2.add_hline(
            y=100,
            line_dash="dash",
            line_color="red",
            annotation_text="Média (100)"
        )
        
        fig2.update_layout(
            title='Índice de Sazonalidade Mensal (Base 100 = Média)',
            xaxis_title='Mês',
            yaxis_title='Índice',
            height=500
        )
        
        fig2.show()
    
    print("\n✅ Análise de séries temporais concluída!")

else:
    print("⚠️ Dados insuficientes para análise temporal")